#AGS1

##0 Setup

In [ ]:
!pip install latextable
!pip install names
!pip install num2words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for latextable: filename=latextable-0.3.0-py3-none-any.whl size=7254 sha256=cadfdef49aa7cca47f3aadeacdcc2e52a6c084eac9f30e2e3518d78678f3c7cd
  Stored in directory: /root/.cache/pip/wheels/a3/8d/a3/41e5c8fb7490c6e93ceb64ca1740f172b7ea837fe42ac41ce0
Successfully built latextable
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 789 kB 3.8 MB/s 
  Created wheel for names: filename=names-0.3.0-py3-none-any.whl size=803699 sha256=8eb0b27a739d0ca91019127219027c649e3f448de51247d2802b119971fee4ea
  Stored in directory: /root/.cache/pip/wheels/05/ea/68/92f6b0669e478af9b7c3c524520d03050089e034edcc775c2b
Successfully built names
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 4.3 MB/s 
  Created whee

In [ ]:
import random
from random import randint, choice, sample

import latextable
from texttable import Texttable

from sympy import *

variables = 'a b c d m n p q r s t w x y'
variables = symbols(variables, real=True)
variables += (Symbol('z'),)
a,b,c,d,m,n,p,q,r,s,t,w,x,y,z = variables

###Functions and sequences

In [ ]:
class LinFunc():
    """
    Represent a linear function.

    Attributes
    ----------
        slope : number (int, float, or Rational)
        intercept : number
        variable : sympy symbol, default=x
        label : str, default='f'
        expr : sympy expression
        domain : sympy set, default=S.Reals
        precision : int, default=2
    """

    def __init__(self, slope, pt, variable=x, label='f', domain=S.Reals, precision=2):
        """
        Construct linear function object from slope and y-intercept.

        Parameters
        ----------
            slope : number (int, float, or Rational)
            pt : number or list
            variable : sympy symbol, default=x
            label : str, default='f'
            domain : sympy set, default=S.Reals
            precision : int, default=2
        """
        
        self.slope, pt = sympify([slope, pt])
        
        self.intercept = pt[1] - pt[0]*self.slope if type(pt)==list else pt
        self.expr = self.slope*variable + self.intercept

        self.variable = variable
        self.label = label
        self.domain = domain
        self.precision = precision
    
    def __str__(self):
        """Return LaTeX formatted slope-intercept form."""
        
        if self.slope in [0,1]:
            slopeInt = ''
        elif self.slope == -1:
            slopeInt = '-'
        else:
            slopeInt = latexify(self.slope, self.precision)
        slopeInt += latex(self.variable) if self.slope!=0 else '' 
        
        intercept = latexify(self.intercept, self.precision)
        if self.intercept>0 and self.slope!=0:
            slopeInt += ' + '
        if self.intercept!=0 or self.slope==0:
            slopeInt += intercept
            
        return slopeInt
    
    @classmethod
    def fromPts(cls, pt1, pt2, variable=x, label='f', domain=S.Reals, precision=2):
        """
        Construct linear function object from two ordered pairs.

        Parameters
        ----------
            pt1, pt2 : list or tuple
            variable : sympy symbol, default=x
            label : str, default='f'
        """

        pt1, pt2 = sympify([pt1, pt2])
        
        slope = (pt2[1] - pt1[1])/(pt2[0] - pt1[0])
        
        if pt1[0]==0:
            return cls(slope, pt1[1], variable, label, domain, precision)
        elif pt2[0]==0:
            return cls(slope, pt2[1], variable, label, domain, precision)
        else:
            return cls(slope, pt1, variable, label, domain, precision)

    def subs(self, input):
        """Return evaluation of function at input."""

        return self.expr.subs(self.variable, sympify(input))

    def subSet(self, preimage):
        """Return image of set."""

        return ImageSet(Lambda(self.variable, self.expr), preimage).intersect(S.Reals)

    def getRange(self):
        """Return range of function."""

        return self.subSet(self.domain)
        
    def solve(self, output):
        if self.slope == 0:
            return self.domain if output==self.intercept else None
        else:
            return (sympify(output)-self.intercept)/self.slope

    def getSlopeInt(self, notation='func'):
        """Return LaTeX formatted slope-intercept form."""

        if notation == 'func':
            return f'{self.label}({self.variable}) = ' + str(self)
        else:
            return notation + '=' + str(self)

    def getPtSlope(self, pt=1, notation='func'):
        """
        Return LaTeX formatted point-slope form.
        
        Parameters
        ---------
            pt : number or list/tuple
                Independent variable value or ordered pair.
            notation : string, default='func'
                LHS of equality, default returns function notation.
        """

        pt = sympify(pt)
        if type(pt)!=list:
            pt = [pt] + [self.subs(pt)]
        
        ptTex = latexify(pt, self.precision)
        
        if self.slope in [0,1]:
            ptSlope = ''
        elif self.slope == -1:
            ptSlope = '-'
        else:
            ptSlope = latexify(self.slope, self.precision)
            
        if self.slope!=0:
            ptSlope += r'\left(' + latex(self.variable)
            ptSlope += '+' + ptTex[0][1:] if pt[0]<0 else '-' + ptTex[0]
            ptSlope += r'\right)'
        
        if pt[1]>0 and self.slope!=0:
            ptSlope += ' + '
        if pt[1]!=0 or self.slope==0:
            ptSlope += ptTex[1]
        
        if notation == 'func':
            return fr'{self.label}({self.variable}) = ' + ptSlope
        else:
            return notation + '=' + ptSlope
            
    def getStdForm(self, notation='y', mult=1):
        if self.slope == 0:
            mult = nsimplify(self.intercept).q
        elif self.intercept == 0:
            mult = nsimplify(self.slope).q
        else:
            mult *= abs(lcm(nsimplify(self.slope).q, nsimplify(self.intercept).q))
        mult = -mult if self.slope<=0 else mult
        
        stdForm = latex(nsimplify(mult*self.slope*self.variable - mult)) + notation
        stdForm += '=' + latex(nsimplify(-mult*self.intercept))
        
        return stdForm
        
    def getTable(self, nums, vals=True, vertical=True, labels=None):
        """
        Return LaTeX formatted table at given positions with or without blanks.
        
        Parameters
        ----------
            nums : list of ints
            vals : bool or list of bools or nonempty/empty strings ('')
            vertical : bool, default=True
            labels : list, default=None
                Defaults are independent variable and function notation.
        """
        
        vals = len(nums)*[vals] if type(vals)==bool else vals
        images = [signify(latexify(self.subs(jj), self.precision)) for jj in nums]
        images = [jj if vals[count] else r'\phantom{' + jj + r'}' for count,jj in enumerate(images)]
        
        nums = sympify(nums)
        nums = [signify(latexify(jj, self.precision)) for jj in nums]
        
        if labels == None:
            labels = [signify(str(self.variable)), fr'${self.label}({self.variable})$']
        if vertical:
            header = labels
            rows = list(zip(nums,images))
        else:
            header = [labels[0]] + list(nums)
            rows = [[labels[1]] + images]
        
        return tableGenerator(header, rows)

class ExpFunc():
    """
    Represent an (unshifted) exponential function.

    Attributes
    ----------
        base : number (int, float, or Rational)
        intercept : number
        variable : sympy symbol, default=x
        label : str, default='f'
        expr : sympy expression
        domain : sympy set, default=S.Reals
        precision : int, default=2
    """

    def __init__(self, base, pt, variable=x, label='f', domain=S.Reals, precision=2):
        """
        Construct exponential function object from base and y-intercept.

        Parameters
        ----------
            base : number (int, float, or Rational)
            intercept : number
            variable : sympy symbol, default=x
            label : str, default='f'
            domain : sympy set, default=S.Reals
            precision : int, default=2
        """

        self.base, pt = sympify([base, pt])
        
        self.intercept = pt[1]/Pow(self.base,pt[0]) if type(pt)==list else pt
        self.expr = self.intercept*(self.base**variable)
        
        self.variable = variable
        self.label = label
        self.domain = domain
        self.precision = precision
    
    def __str__(self):
        """Return LaTeX formatted base-intercept form."""

        if self.intercept == 1:
            baseInt = ''
        elif self.intercept == -1:
            baseInt = '-'
        else:
            baseInt = latexify(self.intercept, self.precision)
            if type(self.base)==Integer and self.base>0:
                baseInt += r' \cdot '
            
        if type(self.base)==Float or abs(self.base.q)>999:
            base = latexify(self.base, self.precision)
            baseInt += fr'({base})^{self.variable}'
        else:            
            baseInt += latex(self.base**self.variable)
            
        return baseInt
    
    @classmethod
    def fromPts(cls, pt1, pt2, variable=x, label='f', domain=S.Reals, precision=2):
        """
        Construct exponential function object from two ordered pairs.

        Parameters
        ----------
            pt1, pt2 : list or tuple
            variable : sympy symbol, default=x
            label : str, default='f'
        """
        
        pt1, pt2 = sympify([pt1, pt2])
        
        base = Pow(pt2[1]/pt1[1], 1/(pt2[0]-pt1[0]))
        
        if pt1[0]==0:
            return cls(base, pt1[1], variable, label, domain, precision)
        elif pt2[0]==0:
            return cls(base, pt2[1], variable, label, domain, precision)
        else:
            return cls(base, pt1, variable, label, domain, precision)

    def subs(self, input):
        """Return evaluation of function at input."""

        return self.expr.subs(self.variable, sympify(input))

    def subSet(self, preimage):
        """Return image of set."""

        return ImageSet(Lambda(self.variable, self.expr), preimage).intersect(S.Reals)

    def getRange(self):
        """Return range of function."""

        return self.subSet(self.domain)
        
    def solve(self, output):
        """Return preimage of given output."""
        
        return log(sympify(output)/self.intercept, self.base)
    
    def getBaseInt(self, notation='func'):
        """Return LaTeX formatted base-intercept form."""

        if notation == 'func':
            return fr'{self.label}({self.variable}) = ' + str(self)
        else:
            return notation + '=' + str(self)

    def getPtBase(self, pt=1, notation='func'):
        """
        Return LaTeX formatted point-base form.
        
        Parameter
        ---------
            pt : number or list/tuple
                Independent variable value or ordered pair.
            notation : string, default='func'
                LHS of equality, default returns function notation.
        """

        pt = sympify(pt)
        if type(pt)!=list:
            pt = [pt] + [self.subs(pt)]
         
        if pt[1] == 1:
            ptBase = ''
        elif pt[1] == -1:
            ptBase = '-'
        else:
            ptBase = latexify(pt[1], self.precision)
            if type(self.base)==Integer and self.base>0:
                ptBase += r' \cdot '
            
        if type(self.base)==Float or abs(self.base.q)>999:
            base = latexify(self.base, self.precision)
            ptBase += fr'({base})^' + r'{' + latex(self.variable-pt[0]) + r'}'
        else:
            ptBase += latex(self.base**(self.variable - pt[0]))
            
        if notation == 'func':
            return fr'{self.label}({self.variable}) = ' + ptBase
        else:
            return notation + '=' + ptBase
        
    def getTable(self, nums, vals=True, vertical=True, labels=None):
        """
        Return LaTeX formatted table at given positions with or without blanks.
        
        Parameters
        ----------
            nums : list of ints
            vals : bool or list of bools or nonempty/empty strings ('')
            vertical : bool, default=True
            labels : list, default=None
                Defaults are independent variable and function notation.
        """
        
        vals = len(nums)*[vals] if type(vals)==bool else vals
        images = [signify(latexify(self.subs(jj), self.precision)) for jj in nums]
        images = [jj if vals[count] else r'\phantom{' + jj + r'}' for count,jj in enumerate(images)]
        
        nums = sympify(nums)
        nums = [signify(latexify(jj, self.precision)) for jj in nums]
        
        if labels == None:
            labels = [signify(str(self.variable)), fr'${self.label}({self.variable})$']
        if vertical:
            header = labels
            rows = list(zip(nums,images))
        else:
            header = [labels[0]] + list(nums)
            rows = [[labels[1]] + images]
        
        return tableGenerator(header, rows)

In [ ]:
class QuadFunc():
    """
    Represent a quadratic function.

    Attributes
    ----------
        aa : number (int, float, or Rational)
        bb : number
        cc : number
        
        variable : sympy symbol, default=x
        label : str, default='f'
        expr : sympy expression
        domain : sympy set, default=S.Reals
        precision : int, default=2
    """

    def __init__(self, aa, bb, cc, variable=x, label='f', domain=S.Reals, precision=2):
        """
        Construct quadratic function object from coefficients.

        Parameters
        ----------
            aa : number (int, float, or Rational)
            bb : number
            cc : number
            variable : sympy symbol, default=x
            label : str, default='f'
            domain : sympy set, default=S.Reals
            precision : int, default=2
        """
        
        self.aa, self.bb, self.cc = sympify([aa, bb, cc])
        
        self.expr = self.aa*variable**2 + self.bb*variable + self.cc

        self.variable = variable
        self.label = label
        self.domain = domain
        self.precision = precision
    
    def __str__(self):
        """Return LaTeX formatted standard form."""
        
        if self.aa in [0,1]:
            stdForm = ''
        elif self.aa == -1:
            stdForm = '-'
        else:
            stdForm = latexify(self.aa, self.precision)
        stdForm += latex(self.variable**2) if self.aa!=0 else '' 
        
        bb = latexify(self.bb, self.precision)
        if self.bb>0 and self.aa!=0:
            stdForm += ' + '
        if self.bb == -1:
            stdForm += ' - '
        elif self.bb not in [0,1]:
            stdForm += bb
        stdForm += latex(self.variable) if self.bb!=0 else ''
            
        cc = latexify(self.cc, self.precision)
        if self.cc>0 and (self.aa!=0 or self.bb!=0):
            stdForm += ' + '
        if self.cc!=0 or (self.aa==0 and self.bb==0):
            stdForm += cc
            
        return stdForm
    
    """@classmethod"""
    # def fromPts(cls, pt1, pt2, variable=x, label='f', domain=S.Reals, precision=2):
    #     """
    #     Construct linear function object from two ordered pairs.

    #     Parameters
    #     ----------
    #         pt1, pt2 : list or tuple
    #         variable : sympy symbol, default=x
    #         label : str, default='f'
    #     """

    #     pt1, pt2 = sympify([pt1, pt2])
        
    #     slope = (pt2[1] - pt1[1])/(pt2[0] - pt1[0])
        
    #     if pt1[0]==0:
    #         return cls(slope, pt1[1], variable, label, domain, precision)
    #     elif pt2[0]==0:
    #         return cls(slope, pt2[1], variable, label, domain, precision)
    #     else:
    #         return cls(slope, pt1, variable, label, domain, precision)

    def subs(self, input):
        """Return evaluation of function at input."""

        return self.expr.subs(self.variable, sympify(input))

    def subSet(self, preimage):
        """Return image of set."""

        return ImageSet(Lambda(self.variable, self.expr), preimage).intersect(S.Reals)

    def getRange(self):
        """Return range of function."""

        return self.subSet(self.domain)
        
    # def solve(self, output):
        # if self.slope == 0:
            # return self.domain if output==self.intercept else None
        # else:
            # return (sympify(output)-self.intercept)/self.slope

    def getStdForm(self, notation='func'):
        """Return LaTeX formatted standard form."""

        if notation == 'func':
            return f'{self.label}({self.variable}) = ' + str(self)
        else:
            return notation + '=' + str(self)

    # def getPtSlope(self, pt=1, notation='func'):
        # """
        # Return LaTeX formatted point-slope form.
        
        # Parameter
        # ---------
            # pt : number or list/tuple
                # Independent variable value or ordered pair.
        # """

        # pt = sympify(pt)
        # if type(pt)!=list:
            # pt = [pt] + [self.subs(pt)]
        
        # ptTex = latexify(pt, self.precision)
        
        # if self.slope in [0,1]:
            # ptSlope = ''
        # elif self.slope == -1:
            # ptSlope = '-'
        # else:
            # ptSlope = latexify(self.slope, self.precision)
            
        # if self.slope!=0:
            # ptSlope += r'\left(' + latex(self.variable)
            # ptSlope += '+' + ptTex[0][1:] if pt[0]<0 else '-' + ptTex[0]
            # ptSlope += r'\right)'
        
        # if pt[1]>0 and self.slope!=0:
            # ptSlope += ' + '
        # if pt[1]!=0 or self.slope==0:
            # ptSlope += ptTex[1]
        
        # if notation == 'func':
            # return fr'{self.label}({self.variable}) = ' + ptSlope
        # else:
            # return notation + '=' + ptSlope
        
    def getTable(self, nums, vals=True, vertical=True, labels=None):
        """
        Return LaTeX formatted table at given positions with or without blanks.
        
        Parameters
        ----------
            nums : list of ints
            vals : bool or list of bools or nonempty/empty strings ('')
        """
        
        vals = len(nums)*[vals] if type(vals)==bool else vals
        images = [signify(latexify(self.subs(jj), self.precision)) for jj in nums]
        images = [jj if vals[count] else r'\phantom{' + jj + r'}' for count,jj in enumerate(images)]
        
        nums = sympify(nums)
        nums = [signify(latexify(jj, self.precision)) for jj in nums]
        
        if labels == None:
            labels = [signify(str(self.variable)), fr'${self.label}({self.variable})$']
        if vertical:
            header = labels
            rows = list(zip(nums,images))
        else:
            header = [labels[0]] + list(nums)
            rows = [[labels[1]] + images]
        
        return tableGenerator(header, rows)

In [ ]:
class PWFunc():
    """
    Represent a piecewise function.

    Attributes
    ----------
        variable : sympy symbol, default=x
        label : str, default='f'
        expr : sympy expression
        domain : sympy set
    """

    def __init__(self, *pairs, variable=x, label='f'):
        """
        Construct piecewise function object from expr-interval pairs.

        Parameters
        ----------
            pairs : array of expr-interval pairs
            variable : sympy symbol, default=x
            label : str, default='f'
        """

        self.variable = variable
        self.label = label
        
        exprs, intervals = zip(*pairs)
        intervals = list(map(makeInterval, intervals))
        self.pairs = list(zip(exprs, intervals))
        
        pieces = [[row[0], Contains(variable, row[1])] for row in self.pairs]
        self.expr = Piecewise(*pieces)
        
        self.domain = Union(*intervals)
    
    def __str__(self):
        """Return LaTeX."""
        # TODO: relational instead of intervals

        return fr'{self.label}({self.variable}) = ' + latex(self.expr)

    def subs(self, input):
        """Return evaluation of function at input."""

        return nsimplify(self.expr.subs(self.variable, input))

    def subSet(self, preimage):
        """Return image of set."""
        
        image = S.EmptySet
        for expr, interval in self.pairs:
            image = Union(image, ImageSet(Lambda(self.variable,expr),interval.intersect(preimage)))

        return image.intersect(S.Reals)

    def getRange(self):
        """Return range of function."""

        image = S.EmptySet
        for expr, interval in self.pairs:
            image = Union(image, ImageSet(Lambda(self.variable,expr),interval))

        return image.intersect(S.Reals)
        
    def solve(self, output):
        """Return preimage of given output."""
        solns = []
        
        for expr, interval in self.pairs:
            answers = solve(expr-output, self.variable)
            for ans in answers:
                if ans in interval:
                    solns.append(ans)
        
        return list(set(solns))
    
    def getLatex(self):
        """Return LaTeX."""

        return str(self)

    def getFeats(self, feat='all', expr='latex'):           # TODO: INC/DEC
        features = dict()
        if feat in ['domain','all']:
            features['domain'] = self.domain
        if feat in ['range','all']:
            features['range'] = self.getRange()
        # if feat in ['change','all']:
        #     features['inc'] = 
        #     features['dec'] = 
        #     features['const'] = 
        if feat in ['extrema','all']:
            rangeset = func.getRange()
            features['mins'] = [func.solve(rangeset.inf), rangeset.inf]
            features['maxes'] = [func.solve(rangeset.sup), rangeset.sup]

        return latexify(features) if expr=='latex' else features

In [ ]:
class ArithSeq(LinFunc):
    """
    Represent an arithmetic sequence.

    Attributes
    ----------
        common : number (int, float, or Rational)
            Common difference.
        start : number
            First term (a_1 or f(1)).
        label : str, default='f'
        func : LinFunc
        terms : dict
    """

    def __init__(self, common, pt, variable=n, label='f', domain=S.Integers, precision=2):
        """
        Construct arithmetic sequence object from common difference and first term.

        Parameters
        ----------
            common : number (int, float, or Rational)
                Common difference.
            pt : number or list
                First term (a_1 or f(1)).
            label : str, default='f'
        """
        
        super().__init__(common, pt, variable, label, domain, precision)
        
        self.common = self.slope
        self.start = self.subs(1)
        self.terms = {0: self.intercept, 1: self.start, 2: self.subs(2), 3: self.subs(3)}
    
    def findTerm(self, num: int):
        """Return term at given index."""

        if num in self.terms.keys():
            return self.terms[num]
        else:
            return self.subs(num)
    
    def findTermNum(self, end):
        """Return index of given term."""

        if end in self.terms.values():
            return list(self.terms.keys())[list(self.terms.values()).index(end)]
        else:
            return self.solve(end)
    
    def getTerms(self, num: int, startnum=0):
        """Return dict of terms."""

        minNum = min(self.terms.keys())
        maxNum = max(self.terms.keys())
        if num > maxNum:
            for jj in range(maxNum+1, num+1):
                self.terms[jj] = self.terms[jj-1] + self.common
        if startnum < minNum:
            for jj in range(minNum-1, startnum-1, -1):
                self.terms[jj] = self.terms[jj+1] - self.common
        return self.terms
    
    def getExplicit(self, term=1):
        """
        Return LaTeX formatted explicit representation.
        
        Parameter
        ---------
            term : number or list/tuple
                Index of term or ordered pair.
        """

        return self.getSlopeInt() if term==0 else self.getPtSlope(term)
    
    def getRecursive(self, startnum=1):
        """
        Return LaTeX formatted recursive representation.

        Parameter
        ---------
            startnum : int
        """

        term = self.findTerm(startnum) if startnum!=1 else self.start
        recursive = fr'{self.label}(n) = {self.label}(n-1)'
        if self.common >= 0:
            recursive += ' + ' + latexify(self.common, self.precision)
        else:
            recursive += latexify(self.common, self.precision)
        recursive += fr',\quad {self.label}({startnum}) = {latexify(term, self.precision)}'
        return recursive
        
    def getSeqStr(self, nums=[1,2,3]):
        """
        Return LaTeX formatted list at given positions with or without blanks.
        
        Parameter
        ---------
            nums : list of ints or empty strings ('')
        """
        
        blank = r'\underline{\hspace{4mm}}'
        result = [latexify(self.findTerm(jj), self.precision) if type(jj)==int else blank for jj in nums]
        
        return (r',\ ').join(result) + r', \ldots'

class GeoSeq(ExpFunc):
    """
    Represent a geometric sequence.

    Attributes
    ----------
        common : number (int, float, or Rational)
            Common ratio.
        start : number
            First term (a_1 or f(1)).
        label : str, default='f'
        func : ExpFunc
        terms : dict
    """

    def __init__(self, common, pt, variable=n, label='f', domain=S.Integers, precision=2):
        """
        Construct geometric sequence object from common ratio and first term.

        Parameters
        ----------
            common : number (int, float, or Rational)
                Common ratio.
            start : number
                First term (a_1 or f(1)).
            label : str, default='f'
        """
        
        super().__init__(common, pt, variable, label, domain, precision)

        self.common = self.base
        self.start = self.subs(1)
        self.terms = {0: self.intercept, 1: self.start, 2: self.subs(2), 3: self.subs(3)}
    
    def findTerm(self, num: int):
        """Return term at given index."""

        if num in self.terms.keys():
            return self.terms[num]
        else:
            return self.subs(num)
    
    def findTermNum(self, end):
        """Return index of given term."""

        if end in self.terms.values():
            return list(self.terms.keys())[list(self.terms.values()).index(end)]
        else:
            return self.solve(end)
    
    def getTerms(self, num: int, startnum=0):
        """Return list of terms."""

        minNum = int(min(self.terms.keys()))
        maxNum = int(max(self.terms.keys()))
        if num > maxNum:
            for jj in range(maxNum+1, num+1):
                self.terms[jj] = self.terms[jj-1]*self.common
        if startnum < minNum:
            for jj in range(minNum-1, startnum-1, -1):
                self.terms[jj] = Rational(self.terms[jj+1],self.common)
        return self.terms
    
    def getExplicit(self, term=1):
        """
        Return LaTeX formatted explicit representation.
        
        Parameter
        ---------
            term : number or list/tuple
                Index of term or ordered pair.
        """

        return self.getBaseInt() if term==0 else self.getPtBase(term)
    
    def getRecursive(self, startnum=1):
        """
        Return LaTeX formatted recursive representation.

        Parameter
        ---------
            startnum : int
        """

        term = self.findTerm(startnum) if startnum!=1 else self.start
        recursive = fr'{self.label}(n) = {latexify(self.common, self.precision)}\cdot {self.label}(n-1)'
        recursive += fr',\quad {self.label}({startnum}) = {latexify(term, self.precision)}'
        return recursive
        
    def getSeqStr(self, nums=[1,2,3]):
        """
        Return LaTeX formatted list at given positions with or without blanks.
        
        Parameter
        ---------
            nums : list of ints or empty strings ('')
        """
        
        blank = r'\underline{\hspace{4mm}}'
        result = [latexify(self.findTerm(jj), self.precision) if type(jj)==int else blank for jj in nums]
        
        return (r',\ ').join(result) + r', \ldots'


###Generators and printers

In [ ]:
def chooseFloat(package):
  floatA = random.choice(package)
  return floatA

def getFloat(min=-10,max=10,roundby=2):
  return round(random.uniform(min, max), roundby)

def getInt(min= -4, max = 4, exclude=[0]):
  intA = random.choice([x for x in range(min,max+1) if x not in exclude])
  return sympify(intA)

def getVar(exclude=None):
  return random.choice([vrbl for vrbl in variables if vrbl not in exclude])

def getFrac(denommin= -4, denommax= 4, numermin = 1, numbermax = 4 , exclude = 0):# exclude 1 in denom later
  top = random.choice([x for x in range(numermin,numbermax+1) if x != exclude])
  bot = random.choice([x for x in range(denommin,denommax+1) if x != exclude])
  return latex(simplify(Rational(top,bot),evaluate=False))

"""###Table"""

def tableGenerator(header, rows, cols_align=None, cols_valign=None, expr='latex', table_env=False, stretch=1.5):
    """
    Generate a table.

    Parameters
    ----------
        header : list or tuple
        rows : 2-dimensional array
        cols_align : list or tuple, default=None
            List of column alignments ('l','c', or 'r'); if ``None``, all set to 'c'.
        expr : str, default='latex'
            LaTeX formatted.
        table_env : boolean, default=True
            Enclose in floating table environment (\begin{table}...).

    Returns
    -------
        str
    """

    table = Texttable()
    if cols_align == None:
        cols_align = len(header)*['c']
    if cols_valign == None:
        cols_valign = len(header)*['m']
    table.set_cols_align(cols_align)
    table.set_cols_valign(cols_valign)
    table.add_rows([header] + rows)

    if expr == 'latex':
        table = latextable.draw_latex(table)
        if not table_env:
            table = table.replace('\\begin{table}\n','').replace('\n\\end{table}','')
        if stretch == 1:
            return table
        else:
            return r'{\renewcommand{\arraystretch}' + brackify(stretch) + table + r'}'
    else:
        return table.draw()
        
def makeInterval(string):
    if type(string)==Interval:
        return string
    left_open = (string[0]=='(')
    right_open = (string[-1]==')')
    start, end = sympify(string[1:-1].split(','))
    return Interval(start, end, left_open, right_open)

In [ ]:
def signify(data):
    if type(data)==list:
        return [signify(jj) for jj in data]
    else:
        return '$' + str(data) + '$'
    
def latexify(data, precision=2):
    if type(data)==list:
        return [latexify(num, precision) for num in data]
    elif type(data)==dict:
        return {k:latexify(v, precision) for k,v in data.items()}
    elif type(data) in [Integer,int]:
        return f'{int(data):,}'
    elif type(data) in [Float,float]:
        return f'{data:,.{precision}f}'
    elif type(data)==Rational and abs(data.q)>999:
        return f'{Float(data):,.{precision}f}'
    elif type(data)==Mul:
        a,b,c = Wild('a'),Wild('b'),Wild('c')
        data = data.match(a*b**c)
        data = data[a]*UnevaluatedExpr(real_root(data[b],data[c].q))**data[c].p
        return latex(data)
    elif type(data)==str:
        return data
    else:
        return latex(data)

def brackify(data):
    return r'{' + str(data) + r'}'
        
def minipagify(left, right, vspace='0.25in'):
    string = '\\begin{minipage}[t]{0.5\\textwidth}\n'
    string += left + '\n\\end{minipage}\n'
    string += '\\begin{minipage}[t]{0.5\\textwidth}\n'
    string += right + '\n\\end{minipage}\n\n'
    string += f'\\vspace{brackify(vspace)}\n'
    return string

###Graphers

In [ ]:
def startGraph(xmin=-10,xmax=10,ymin=-10,ymax=10):
    xstart = 5*(xmin//5)
    ystart = 5*(ymin//5)
    xtick = brackify(f'{xstart},{xstart+5},...,{xmax+5}')
    ytick = brackify(f'{ystart},{ystart+5},...,{ymax+5}')

    tex = r'\begin{tikzpicture}\begin{axis}[mmt axis style,'
    tex += fr'xmin={xmin},xmax={xmax},xtick={xtick},ymin={ymin},ymax={ymax},ytick={ytick},]'
    return tex

def endGraph():
    return r'\end{axis}\end{tikzpicture}'

def emptyGraph(xmin=-10,xmax=10,ymin=-10,ymax=10):
    return startGraph(xmin,xmax,ymin,ymax) + endGraph()

def drawPt(pt, draw='black', fill='black', shape='circle', size='2pt'):
    return fr'\fill[draw={draw},fill={fill}] (axis cs: {pt[0]},{pt[1]}) {shape}({size});'

def drawScatter(pts, draw='black', fill='black', shape='', size='2pt'):
    """
    Return LaTeX to draw list of points on a graph.

    Parameters
    ----------
        pts : list
            List of coordinate pairs.
        draw : string, default='black'
            Color of mark boundaries.
        fill : string, default='black'
            Color of mark fills.
        shape : string, default=''
            Shape of marks, default='' gives circle.
                Other options include 'triangle','square','diamond,'star','otimes'.
        size : string, default='2pt'
    """

    pts = ' '.join([str(tuple(jj)) for jj in pts])
    return fr'\addplot[only marks,draw={draw},fill={fill},mark={shape}*,mark size={size}] coordinates {brackify(pts)};'
    
def drawLinear(expr,xmin,xmax,color='black',style='solid'):
    string = fr'\addplot[{style},domain={xmin}:{xmax}, color={color},]{brackify(expr)};'
    return string

def drawCurve(expr, inMin, inMax, LHS='y', color='black', style='solid', width='thick', label=None, fillName=None):
    """
    Return LaTeX to draw a curve on a graph.

    Parameters
    ----------
        expr : string or sympy expression
            Right-hand side of equation
        inMin : number
            Minimum value of independent variable.
        inMax : number
            Maximum value of independent variable.
        LHS : string, default='y'
            Left-hand side of equation.
        color : string, default='black'
        style : string, default='solid'
        width : string, default='thick'
        label : list or string, default=None
            If list, format as [<label>, <rel. pos.>, <angle>] where rel. pos.
                is relative position as a float in [0,1] and angle is given in degrees.
            If string, default rel. pos.=0.7 and default angle=45.
        fillName : string, default=None
            Name of curve used for fillbetween library (needed only for inequality graphers).
    """
    
    string = fr'\addplot[{color},{style},{width},domain={inMin}:{inMax},samples=100'
    string += r'] ' if fillName==None else fr',name path={fillName}] '

    variables = sympify(expr).free_symbols
    expr = str(expr).replace('**','^')

    if (variables=={x}) or (LHS in ['y',y]):        # Function of x
        string += brackify(expr)
    elif (variables=={y}) or (LHS in ['x',x]):      # Function of y
        expr = expr.replace('y','x')
        string += f'({brackify(expr)},{brackify(x)})'

    if label==None:
        return string + r';'
    else:
        if type(label) != list:
            label = [label, 0.7, 45]
        return string + fr' node[pos={label[1]},pin={label[2]}:{signify(label[0])}]' + r' {};'

def drawSlopeTri(pt1, pt2):
    string = fr'\draw[dashed] (axis cs: {pt1[0]},{pt1[1]}) -- (axis cs: {pt1[0]},{pt2[1]});'
    string += fr'\draw[dashed] (axis cs: {pt1[0]},{pt2[1]}) -- (axis cs: {pt2[0]},{pt2[1]});'
    return string

def shadeRegion(xmin, xmax, lower, upper, color='blue!30', shading='opacity=0.5'):
    domain = fr'domain={xmin}:{xmax}'
    string = fr'\addplot[{color},{shading}] fill between'
    string += fr'[of={lower} and {upper},soft clip={brackify(domain)}];'
    return string

###Word problems

In [ ]:
import names, pandas
from num2words import num2words

DIFF, RATIO, SEQ_1, SEQ_2, SEQ_3 = symbols('DIFF RATIO SEQ_1 SEQ_2 SEQ_3')
INTA, INTB, FLOATA, FLOATB = symbols('INTA INTB FLOATA FLOATB')

Lin_probs = pandas.read_csv('Lin_probs.csv')
Lin_probs = Lin_probs[['Question','Answer1','Answer2','Controller','In','Out']]

Exp_probs = pandas.read_csv('Exp_probs.csv')
Exp_probs = Exp_probs[['Question','Answer1','Answer2','Controller','In','Out']]

def getControls(controls):
    params = dict()

    for word in controls:
        word, range = word.split('=')
        range = range.replace('[','').replace(']','').split(',')
        mnm, mxm = sympify(range[:2])
        params[word] = getFloat(mnm,mxm) if type(mnm)==Float else getInt(mnm,mxm)

    return params

def getParam(word, params, kind):
    if word == 'LASTNAME':
        return names.get_last_name()
    elif word == 'FIRSTNAME':
        return names.get_first_name()
    elif word == 'FIRSTNAME_MALE':
        return names.get_first_name(gender='male')
    elif word == 'FIRSTNAME_FEMALE':
        return names.get_first_name(gender='female')
    elif word == 'SEQ_2':
        if kind == 'lin':
            return params['SEQ_1'] + params['DIFF']
        elif kind == 'exp':
            return params['SEQ_1']*params['RATIO']
    elif word == 'SEQ_3':
        if kind == 'lin':
            return 2*params['SEQ_2'] - params['SEQ_1']
        elif kind == 'exp':
            return params['SEQ_2']**2/params['SEQ_1']
    elif word == 'MULT_PHRASE':
        if 'RATIO' not in params.keys():
            params['RATIO'] = randint(2,5)
        phrase = 'twice' if params['RATIO']==2 else num2words(params['RATIO']) + ' times'
        return phrase
    elif word == 'GROWTH_PHRASE':
        growths = {2: 'doubles', 3: 'triples', 4: 'quadruples'}
        if 'RATIO' not in params.keys():
            params['RATIO'] = randint(2,5)
        if params['RATIO'] in growths.keys():
            phrase = growths[params['RATIO']]
        else:
            phrase = 'grows by a factor of ' + num2words(params['RATIO'])
        return phrase

def getWordValue(word, params, kind, string=True):
    symbol1 = ''
    symbol2 = ''
    if (not word[0].isalnum()) and word[0]!='(':
        symbol1 = '\\' + '$' if word[0] in ['$','£'] else word[0]
        word = word[1:]
    if (not word[-1].isalnum()) and word[-1]!=')':
        symbol2 = '\\' + word[-1] if word[-1]=='%' else word[-1]
        word = word[:-1]

    if '(' in word:
        word = sympify(word).subs(params.items())
    elif word.isnumeric():
        word = sympify(word)
    else:
        if word not in params.keys():
            params[word] = getParam(word, params, kind)
        word = params[word]
    
    return symbol1 + str(latexify(word)) + symbol2 if string else word

def makeWordProb(kind='lin', expr='latex'):
    if kind == 'lin':
        data = Lin_probs.loc[randint(0,len(Lin_probs)-1)]
    elif kind == 'exp':
        data = Exp_probs.loc[randint(0,len(Exp_probs)-1)]
    else:
        data = Neither_probs.loc[randint(0,len(Neither_probs)-1)]

    problem = data[0].split()
    solns = data[1:3].copy()
    controls = data[3].split()
    labels = list(data[4:6].values)

    params = getControls(controls)

    expressions = []
    for count,word in enumerate(problem):
        if word.isupper() and len(word)>2:
            if '(' in word:
                expressions.append([count,word])
            else:
                problem[count] = getWordValue(word, params, kind)
    for count,word in expressions:
        problem[count] = getWordValue(word, params, kind)
    
    for jj in [0,1]:
        solns[jj] = solns[jj].split(',')
        if len(solns[jj]) > 1:
            solns[jj] = [ getWordValue(solns[jj][0], params, kind, string=False), getWordValue(solns[jj][1], params, kind, string=False) ]
        else:
            solns[jj] = getWordValue(solns[jj][0], params, kind, string=False)

    return ' '.join(problem), solns, labels

FileNotFoundError: ignored

##1.1.1 Evaluate Equations




---
- startA = [-10,10]
- startB = [-20,20]
- floatA = [ 0.3 , 0.5 , 0.9]
- floatB = [ -9.9,9.9]
- intA = [-4,-3,-2,-1,1,2,3,4]
- intB = [-4,-3,-2,-1,1,2,3,4]
- intC = [-4,-3,-2,-1,1,2,3,4]
- intD = [-4,-3,-2,-1,1,2,3,4]
- intE = [-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,1,2,3,4,5,6,7,8,9,10]
- intF = [-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,1,2,3,4,5,6,7,8,9,10]
- intG =
- intH =
---

### Section 1
####instruction:  Identify	which	of	the	3	possible	numbers	is	the	solution	to	the	equation.

- easy
  - case 1
  - problem : inA*x + intB = intC (x = intD; x = intE ; x = intF)
  - answer :  
  - problem : $3x + 2 = -4 $ ($x=3;x=2;x=-2$)
  - answer : x = -2
  - case 2
  - problem : inA*x + intB*x = intC (x = intD; x = intE ; x = intF)
  - answer :  
  - problem : $2x + x = -3 $ ($x=3;x=2;x=-2$)
  - answer : x = -1
- medium
  - case 1
  - problem : inA*x + intB*x + intC = intD (x = ; x =  ; x = )
  - answer :  x = 
  - problem : 8x + 7x + 3  = 13 (x = -3; x = 2 ; x = 5)
  - answer x = , ,
  - case 2
  - problem : inA(x + intB) + intC = intD (x = ; x =  ; x = )
  - answer :  x = 
  - problem : 2(x + 3)  = 5 (x = -3; x = 2 ; x = 5)
  - answer x = , ,
- hard 
  - case 1
  - problem : intA(intB*x + intC*x) + intD = intE (x = ; x =  ; x = )
  - answer :  x = 
  - problem : 2(x + 7) + 3  = 5 (x = -3; x = 2 ; x = 5)
  - answer x = , ,
  - case 2
  - problem : inA*x + intB*x + intC = intD (x = ; x =  ; x = )
  - answer :  x = 
  - problem : $3(6m-17) = 13 (x = -3; x = 2 ; x = 5)$
  - answer x = , ,


In [ ]:
def Evaluate_Equation_1(difficulty=1, expr="latex"):
  problem = ""
  answer = ""
  if difficulty == 1:   # Easy
    choice = random.randint(1,2)
    intA = getInt()
    intB = getInt( 2 , 4 ) # always positive
    intC = getInt()
    intD = getInt()
    fracA = getFrac()
    if choice == 1:
      answer = latex(simplify(Rational( intC - intB ,intA)))
      print(answer, answer == "\tilde{\infty}")
      while answer == "\tilde{\infty}":
        answer = latex(simplify(Rational( intC - intB ,intA)))
      answerpool = [intD,fracA,answer]
      random.shuffle(answerpool)
      temp = latex(f"{intA} x + {intB} = {intC}")
      problem = f"${temp} , x={answerpool[0]};x={answerpool[1]};x={answerpool[2]}$"
    elif choice == 2:
      answer = latex(simplify(Rational( intC  , intA+intB)))
      print(answer, answer == "\tilde{\infty}")
      while answer == "\tilde{\infty}":
        answer = latex(simplify(Rational( intC  , intA+intB)))
      answerpool = [intD,fracA,answer]
      random.shuffle(answerpool)
      temp = latex(f"{intA} x + {intB} x = {intC}")
      problem = f'${temp} , x={answerpool[0]};x={answerpool[1]};x={answerpool[2]}$'
  elif difficulty == 2 or difficulty == 3:
    choice = random.randint(1,2)
    intA = getInt()
    intB = getInt()
    intC = getInt()
    intD = getInt()
    fracA = getFrac()
    if choice == 1:
      answer = latex(simplify(Rational( intC - intC ,intA + intB)))
      while answer == "\tilde{\infty}":
        answer = latex(simplify(Rational( intC - intC ,intA + intB)))
      answerpool = [intD,fracA,answer]
      random.shuffle(answerpool)
      temp = latex(f"{intA} x + {intB} x + {intC} = {intD}")
      problem = f"${temp}, x={answerpool[0]};x={answerpool[1]};x={answerpool[2]}$"
    elif choice == 2:
      answer = latex(simplify(Rational( intC  , intA+intB)))
      while answer == "\tilde{\infty}":
        answer = latex(simplify(Rational( intC  , intA+intB)))
      answerpool = [intD,fracA,answer]
      random.shuffle(answerpool)
      temp = latex(f"{intA} x + {intB} x = {intC}")
      problem = f'${temp} , x={answerpool[0]};x={answerpool[1]};x={answerpool[2]}$'
  answer = f"x={answer}"
  return problem , answer

for i in range(30):
  problem, answer = Evaluate_Equation_1(1, "latex")
  print(problem, answer)

"""print("medium")
for i in range(3):
    problem, answer = Evaluate_Equations_Section_1("medium", "latex")
    print(problem, answer)

print("hard")
for i in range(3):
    problem, answer = Evaluate_Equations_Section_1("hard", "latex")
    print(problem, answer)
"""

\tilde{\infty} False
$\mathtt{\text{-3 x + 3 x = -4}} , x=1;x=-3;x=\tilde{\infty}$ x=\tilde{\infty}
-3 False
$\mathtt{\text{2 x + 4 = -2}} , x=-3;x=2;x=-2$ x=-3
\frac{3}{5} False
$\mathtt{\text{1 x + 4 x = 3}} , x=-1;x=-1;x=\frac{3}{5}$ x=\frac{3}{5}
- \frac{1}{3} False
$\mathtt{\text{1 x + 2 x = -1}} , x=- \frac{1}{3};x=-3;x=-2$ x=- \frac{1}{3}
3 False
$\mathtt{\text{-3 x + 4 x = 3}} , x=- \frac{1}{4};x=1;x=3$ x=3
\frac{1}{4} False
$\mathtt{\text{-4 x + 3 = 2}} , x=\frac{1}{2};x=\frac{1}{4};x=4$ x=\frac{1}{4}
- \frac{5}{3} False
$\mathtt{\text{3 x + 2 = -3}} , x=-3;x=1;x=- \frac{5}{3}$ x=- \frac{5}{3}
\frac{2}{7} False
$\mathtt{\text{4 x + 3 x = 2}} , x=3;x=-3;x=\frac{2}{7}$ x=\frac{2}{7}
\tilde{\infty} False
$\mathtt{\text{-4 x + 4 x = -2}} , x=2;x=\tilde{\infty};x=-4$ x=\tilde{\infty}
- \frac{5}{2} False
$\mathtt{\text{2 x + 2 = -3}} , x=- \frac{5}{2};x=\frac{3}{4};x=4$ x=- \frac{5}{2}
\frac{3}{4} False
$\mathtt{\text{1 x + 3 x = 3}} , x=-4;x=-1;x=\frac{3}{4}$ x=\frac{3}{4}
0 False


'print("medium")\nfor i in range(3):\n    problem, answer = Evaluate_Equations_Section_1("medium", "latex")\n    print(problem, answer)\n\nprint("hard")\nfor i in range(3):\n    problem, answer = Evaluate_Equations_Section_1("hard", "latex")\n    print(problem, answer)\n'

In [ ]:
#Test---
from random import random, choice
from sympy import sympify
UNARIES = ["%s", "-%s"]
BINARIES = ["%s + %s", "%s - %s"]

PROP_PARANTHESIS = 0.3
PROP_BINARY = 0.7

def generate_expressions(scope, num_exp, num_ops):
    scope = list(scope) # make a copy first, append as we go
    for i in range(num_ops):
        if random() < PROP_BINARY: # decide unary or binary operator
            ex = choice(BINARIES) % (choice(scope), choice(scope))
            if random() < PROP_PARANTHESIS:
                ex = "(%s)" % ex
            scope.append(sympify(ex,evaluate=False))
        else:
            scope.append(choice(UNARIES) % choice(scope))
    return scope[-num_exp:] # return most recent expressions

#scope = [c for c in "abcde"]
scope = range(100)

for expression in generate_expressions(scope, 10, 50):
    print (expression)


38*(-1) + 48 + 65 + 83
-53 + (-1)*15
21 + 36
(-1)*38 + 22 + 48
32 + 52
99 + (-1)*27
46 + 82
82 + 88
22
-53 + (-1)*15


In [ ]:
#Test---
import random

def expr(depth):
    if depth==1 or random.random()<1.0/(2**depth-1): 
        return str(int(random.random() * 100))
    return '(' + expr(depth-1) + random.choice(['+','-','*','/']) + expr(depth-1) + ')'

for i in range(10):
    print (expr(4))

(2+30)
(((21*77)-(67*61))*32)
((1-52)-((14-23)-(58/95)))
((17+(14*72))-75)
(((48/30)-(6/98))+((50-49)/(97*23)))
((57*46)+((20+17)*(57/29)))
90
(((29*65)/(77-65))/31)
((86*(22+55))-73)
(((98-45)*(63/46))/((8-79)+59))


In [ ]:
#Test---
import sys
import random

# dictionary of operator precedence and incidence probability, with an
# evaluator added just for fun.
operators = {
    '^': {'prec': 10, 'prob': .1, 'eval': lambda a, b: pow(a, b)},
    '*': {'prec': 20, 'prob': .2, 'eval': lambda a, b: a*b},
    '/': {'prec': 20, 'prob': .2, 'eval': lambda a, b: a/b},
    '+': {'prec': 30, 'prob': .25, 'eval': lambda a, b: a+b},
    '-': {'prec': 30, 'prob': .25, 'eval': lambda a, b: a-b}}

max_levels = 3
integer_range = (-100, 100)
random.seed()

# A node in an expression tree
class expression(object):
    def __init__(self):
        super(expression, self).__init__()

    def precedence(self):
        return -1

    def eval(self):
        return 0

    @classmethod
    def create_random(cls, level):
        if level == 0:
            is_op = True
        elif level == max_levels:
            is_op = False
        else:
            is_op = random.random() <= 1.0 - pow(level/max_levels, 2.0)

        if is_op:
            return binary_expression.create_random(level)
        else:
            return integer_expression.create_random(level)

class integer_expression(expression):
    def __init__(self, value):
        super(integer_expression, self).__init__()

        self.value = value

    def __str__(self):
        return self.value.__str__()

    def precedence(self):
        return 0

    def eval(self):
        return self.value

    @classmethod
    def create_random(cls, level):
        return integer_expression(random.randint(integer_range[0],
                                                 integer_range[1]))

class binary_expression(expression):
    def __init__(self, symbol, left_expression, right_expression):
        super(binary_expression, self).__init__()

        self.symbol = symbol
        self.left = left_expression
        self.right = right_expression

    def eval(self):
        f = operators[self.symbol]['eval']
        return f(self.left.eval(), self.right.eval())

    @classmethod
    def create_random(cls, level):
        symbol = None

        # Choose an operator based on its probability distribution
        r = random.random()
        cumulative = 0.0
        for k, v in operators.items():
            cumulative += v['prob']
            if r <= cumulative:
                symbol = k
                break

        assert symbol != None

        left = expression.create_random(level + 1)
        right = expression.create_random(level + 1)

        return binary_expression(symbol, left, right)

    def precedence(self):
        return operators[self.symbol]['prec']

    def __str__(self):
        left_str = self.left.__str__()
        right_str = self.right.__str__()
        op_str = self.symbol

        # Use precedence to determine if we need to put the sub expressions in
        # parentheses
        if self.left.precedence() > self.precedence():
            left_str = '('+left_str+')'
        if self.right.precedence() > self.precedence():
            right_str = '('+right_str+')'

        # Nice to have space around low precedence operators
        if operators[self.symbol]['prec'] >= 30:
            op_str = ' ' + op_str + ' '

        return left_str + op_str + right_str

max_result = pow(10, 10)
for i in range(10):
    expr = expression.create_random(0)

    try:
        value = float(expr.eval())
    except:
        value = 'indeterminate'

    print (expr, '=', value)

-10 - -42^83 - 35 - -16 + -31 = 5.366499621007081e+134
-15*-38^81*(-92 - -14*92) = 1.6454843823616902e+132
(-5 + 9)*(-4 - -61)*-10*50/18^48 = -6.365383973955343e-56
65*(-8 - -1)/(-90 - 12) = 4.46078431372549
-23 + -69 + 25 + -50/79 = -67.63291139240506
16/32 + 44/38/70^30 = 0.5
78^(-92 - 78) + (68 - 12)/(73 + -60) = 4.3076923076923075
0*(-25 + 0)/(-27 - -38) = -0.0
57*(28 + -27)/(-76*61 - 91) = -0.012058387983922149
13/-30/-12*-46 = -0.11304347826086956


### Section 2
#### instruction : Determine	the	y-value	of	each	ordered	pair	based	on	the	given	x- value.
- easy
  - case 1
  - problem : $f(x) = 3x + 7  $
  - $(-3 , ) , ( 2 , ) , ( 5 , )$
  - answer : $(-3 , ) , ( 2 , ) , ( 5 , )$
  - case 2
  - problem : $f(x) = \frac{1}{3}x + 7  $
  - answer : $(-3 , ) , ( 2 , ) , ( 5 , )$
- medium
  - case 1
  - problem : $f(x) = 3(6m-17)$
  - answer : $(-3 , ) , ( 2 , ) , ( 5 , )$
  - case 2
  - problem : $f(x) = \frac{2(x+10)}{3}$
  - answer : $(-3 , ) , ( 2 , ) , ( 5 , )$
- hard 
  - case 1
  - problem : $f(x) = 2x^2 + 3x + 5$
  - answer : $(-3 , ),( 2 , ) , ( 5 , )$
  - case 2
  - problem : $f(x) = 2x^2 + 3x + 5$
  - answer : $(-3 , ),( 2 , ) , ( 5 , )$

In [ ]:
def FindYvals(difficulty=1, expr='latex'):

    if difficulty == 1: # easy
        slope = randint(2,9) if randint(0,1) else Rational(1,randint(2,9))
        func = LinFunc(slope, getInt(-9,9))
        mult = S.One

        problem = signify(func.getSlopeInt()) + r' \newline '
    elif difficulty == 2: # medium
        func = LinFunc(randint(2,9), getInt(-15,15))
        mult = sympify(randint(2,7)) if randint(0,1) else Rational(1,randint(2,7))

        problem = fr'{func.label}({func.variable}) = {latex(mult)}\left({str(func)}\right)'
        problem = signify(problem) + r' \newline '
    elif difficulty == 3: # hard
        func = QuadFunc(randint(1,4), randint(-5,5), randint(-9,9))
        mult = sympify(randint(2,5)) if randint(0,1) else Rational(1,randint(2,5))

        problem = fr'{func.label}({func.variable}) = {latex(mult)}\left({str(func)}\right)'
        problem = signify(problem) + r' \newline '
    
    inputs = random.sample(list(range(-10,11)),3)
    inputs.sort()
    problem += fr'$({inputs[0]},_), ({inputs[1]},_), ({inputs[2]},_)$'
    problem = problem.replace('_', r'\underline{\hspace{4mm}}')

    outputs = [func.subs(jj) for jj in inputs]
    outputs = [latexify(mult*jj, func.precision) for jj in outputs]
    if expr=='latex':
        answer = fr'$({inputs[0]}, {outputs[0]}), ({inputs[1]}, {outputs[1]}), ({inputs[2]}, {outputs[2]})$'
    else:
        answer = outputs

    return problem, answer

for jj in range(10):
    problem, answer = FindYvals(3)
    print(problem, r'\\')
    print(answer, r'\\ \\')

$f(x) = 4\left(2x^{2}-2x + 5\right)$ \newline $(1,\underline{\hspace{4mm}}), (4,\underline{\hspace{4mm}}), (9,\underline{\hspace{4mm}})$ \\
$(1, 20), (4, 116), (9, 596)$ \\ \\
$f(x) = 3\left(2x^{2} + 3x + 8\right)$ \newline $(-6,\underline{\hspace{4mm}}), (-1,\underline{\hspace{4mm}}), (6,\underline{\hspace{4mm}})$ \\
$(-6, 186), (-1, 21), (6, 294)$ \\ \\
$f(x) = 4\left(2x^{2}-4x + 1\right)$ \newline $(-1,\underline{\hspace{4mm}}), (6,\underline{\hspace{4mm}}), (9,\underline{\hspace{4mm}})$ \\
$(-1, 28), (6, 196), (9, 508)$ \\ \\
$f(x) = 5\left(4x^{2} + x + 9\right)$ \newline $(-3,\underline{\hspace{4mm}}), (1,\underline{\hspace{4mm}}), (9,\underline{\hspace{4mm}})$ \\
$(-3, 210), (1, 70), (9, 1710)$ \\ \\
$f(x) = 4\left(2x^{2} + 4x + 4\right)$ \newline $(-9,\underline{\hspace{4mm}}), (6,\underline{\hspace{4mm}}), (7,\underline{\hspace{4mm}})$ \\
$(-9, 520), (6, 400), (7, 520)$ \\ \\
$f(x) = \frac{1}{4}\left(x^{2} + 5x\right)$ \newline $(5,\underline{\hspace{4mm}}), (7,\underline{\hspa

---
### Section 3
#### instruction : Given that $f(x) = 8x-3$ and $g(x) = 3x-10$, evaluate the following functions with the indicated values.

- easy
  - case 1
  - problem : linear equation 
  - f(x) = intA * x + intB
  - g(x) = intA * x + intB
  - $f(5)$ = 
  - $g(5)$ =
  - case 2
  - f(x) = intA * x + intB
  - g(x) = intA * x + intB
  - $f(5) = $
  - $g(5) = $
- medium
  - case 1
  - problem : linear equation 
  - f(x) = intA * x + intB
  - g(x) = intA * x + intB
  - $f(5)$ = 
  - $g(5)$ =
  - case 2
  - f(x) = intA * x + intB
  - g(x) = intA * x + intB
  - $f(5) = $
  - $g(5) = $
- hard 
  - case 1
  - problem : linear equation 
  - f(x) = intA * x + intB
  - g(x) = intA * x + intB
  - $f(5)$ = 
  - $g(5)$ =
  - case 2
  - f(x) = intA * x + intB
  - g(x) = intA * x + intB
  - $f(5) = $
  - $g(5) = $


In [ ]:
def EvaluateFunctions(difficulty=1, expr='latex'):
    if difficulty == 1: # easy
        func1 = LinFunc(randint(1,9), randint(1,9))
        func2 = LinFunc(randint(1,9), randint(1,9), label='g')
    elif difficulty == 2: # medium
        func1 = LinFunc(getInt(-9,9), getInt(-9,9))
        func2 = LinFunc(getInt(-9,9), getInt(-9,9), label='g')
    elif difficulty == 3: # hard
        func1 = LinFunc(Rational(getInt(-9,9),randint(2,7)), Rational(getInt(-9,9),randint(2,7)))
        func2 = LinFunc(Rational(getInt(-9,9),randint(2,7)), Rational(getInt(-9,9),randint(2,7)), label='g')

    problem = signify(func1.getSlopeInt()) + r' \newline '
    problem += signify(func2.getSlopeInt()) + r' \newline '

    input = getInt(-9,9)
    problem += fr'${func1.label}({input}) = \qquad,\qquad {func2.label}({input}) = \qquad$'

    outputs = [latexify(func1.subs(input)), latexify(func2.subs(input))]
    if expr == 'latex':
        answer = fr'${func1.label}({input}) = {outputs[0]},\qquad {func2.label}({input}) = {outputs[1]}$'
    else:
        answer = outputs

    return problem, answer

for jj in range(10):
    problem, answer = EvaluateFunctions(3)
    print(problem, r'\\')
    print(answer, r'\\')

$f(x) = -3x + 3$ \newline $g(x) = - \frac{3}{2}x + \frac{2}{3}$ \newline $f(3) = \qquad,\qquad g(3) = \qquad$ \\
$f(3) = -6,\qquad g(3) = - \frac{23}{6}$ \\
$f(x) = \frac{8}{7}x- \frac{9}{2}$ \newline $g(x) = - \frac{2}{3}x + 4$ \newline $f(-2) = \qquad,\qquad g(-2) = \qquad$ \\
$f(-2) = - \frac{95}{14},\qquad g(-2) = \frac{16}{3}$ \\
$f(x) = - \frac{1}{7}x + \frac{7}{6}$ \newline $g(x) = x- \frac{3}{7}$ \newline $f(-2) = \qquad,\qquad g(-2) = \qquad$ \\
$f(-2) = \frac{61}{42},\qquad g(-2) = - \frac{17}{7}$ \\
$f(x) = - \frac{4}{3}x- \frac{5}{7}$ \newline $g(x) = -2x + \frac{3}{2}$ \newline $f(5) = \qquad,\qquad g(5) = \qquad$ \\
$f(5) = - \frac{155}{21},\qquad g(5) = - \frac{17}{2}$ \\
$f(x) = \frac{1}{3}x + 1$ \newline $g(x) = - \frac{7}{2}x + \frac{7}{6}$ \newline $f(4) = \qquad,\qquad g(4) = \qquad$ \\
$f(4) = \frac{7}{3},\qquad g(4) = - \frac{77}{6}$ \\
$f(x) = - \frac{4}{7}x + \frac{1}{2}$ \newline $g(x) = - \frac{9}{7}x + \frac{6}{7}$ \newline $f(9) = \qquad,\qquad g(9) = \qquad

## 1.1.2 Graph The Ordered Pairs




### section1
#### Graph the ordered pairs from the tables on the given graphs. 

- Easy (no ticks given)
  - case 1 - linear increasing
  - problem : (0,4),(2,8),(3,10),5,14)
  - answer : 
  - case 2 - linear decresing
  - problem : (0,12),(4,8),(7,5),(9,3)
  - answer : 
- Medium
  - case 1 - geo increasing
  - problem : (0,2),(2,4),(3,8),(5,32)
  - answer : 
  - case 2 - geo decresing
  - problem : (0,36),(2,18),(4,6),(,3)
  - answer : 
- Hard
  - case 1 - linear 
  - problem : $f(n) = 3n + 5$
  - answer : fill in the table & plot the graph
  - case 2 - geo
  - problem : $f(n) = 3^x$
  - answer : fill in the table & plot the graph

In [ ]:
# #Latex setup code to plot points

# \documentclass[letterpaper,12pt]{article}%

# \usepackage{tikz}
# \usepackage{pgfplots}
# \pgfmathsetseed{\number\pdfrandomseed}
 
# \pgfplotsset{
#     mmt axis style/.style={
#         grid=both,
#         grid style={line width=.1pt, draw=gray!20},
#         major grid style={line width=.2pt,draw=gray!60},
#         axis lines=middle,
#         minor tick num=4,
#         enlargelimits={abs=0.5},
#         unit vector ratio*=1 1 1,
#         axis line style={-latex},
#         ticks=none,
#     },
# } 


# \begin{document}

# \begin{tikzpicture}

# \begin{axis}[
#      mmt axis style,
#     xmin=-1,xmax=10,xtick={10,5,...,-1},
#     ymin=-1,ymax=15,ytick={15,10,...,-1},
# ]

# \fill[black] (axis cs: 3,2) circle(2pt);

# \end{axis}

# \end{tikzpicture}

# \end{document}

In [ ]:
def GraphThePairs(difficulty=1):
    if difficulty == 1: # easy
        vals = True
        if randint(0,1):
            func = LinFunc(randint(1,4), randint(1,4))
        else:
            func = LinFunc(randint(-4,-1), randint(10,14))
    elif difficulty == 2: # medium
        vals = True
        if randint(0,1):
            func = ExpFunc(randint(2,4), randint(1,4))
        else:
            denom = randint(2,4)
            start = randint(1,4)*denom**3
            func = ExpFunc(Rational(1,denom), start)
    elif difficulty == 3: # hard
        vals = False
        if randint(0,1):
            func = LinFunc(randint(1,4), randint(1,4))
        else:
            func = ExpFunc(randint(2,4), randint(1,4))

    ymax = 15 if type(func)==LinFunc else 30
    nums = sample(list(range(0,11)),4)
    nums.sort()
    problem = func.getTable(nums,vals,labels=['$x$','$y$'])
    if difficulty == 3:
        problem = '$y = ' + str(func) + r'$ \newline ' + problem
    problem += emptyGraph(ymax=ymax)

    answer = startGraph(ymax=ymax)
    pts = [drawPt([jj,func.subs(jj)]) for jj in nums]
    answer += ' '.join(pts) + endGraph()
    if difficulty == 3:
        answer = func.getTable(nums,labels=['$x$','$y$']) + answer

    return problem, answer

for jj in range(5):
    problem, answer = GraphThePairs(3)
    print(problem, r'\\')
    print(answer, r'\\')

NameError: ignored

## 1.2.1 Fill In The Sequence






### section 1:
#### Use the given values in the table to determine a pattern and complete the table.

Easy
- case 1
- geo increase [2,3,4]
- start : [-10~10]

| Term |1st | 2nd  |  3rd |  4th | 5th  |  6th | 7th  |  8th |
|---|---|---|---|---|---|---|---|---|
| value | 3 | 6  | 12  | 24  |   |   | | |

- case 2
- linear increase [-4,-3,-2,-1,1,2,3,4]
- start : [-10~10]
  
| Term |1st | 2nd  |  3rd |  4th | 5th  |  6th | 7th  |  8th |
|---|---|---|---|---|---|---|---|---|
| value | 3 | 6  | 9  | 12  |   |   | | |

Medium
- case 1
- geo increase [-4,-3,-2,2,3,4]
- start : [-10~20]

| Term |1st | 2nd  |  3rd |  4th | 5th  |  6th | 7th  |  8th |
|---|---|---|---|---|---|---|---|---|
| value | 3 | -6  | 12  | -24  |   |   | | |

- case 2
- linear increase
- rate of change [-15~15] except 0
- start : [-20~20]
  
| Term |1st | 2nd  |  3rd |  4th | 5th  |  6th | 7th  |  8th |
|---|---|---|---|---|---|---|---|---|
| value | -7 | -4  | -1  | 2  |   |   | | |

Hard 

- case 1
- geo increase [-6,-5,-4,4,5,6]
- start : [-20~20]

| Term |1st | 2nd  |  3rd |  4th | 5th  |  6th | 7th  |  8th |
|---|---|---|---|---|---|---|---|---|
| value | -17 |  102 | 612  | -3672  |   |   | | |

- case 2
- linear increase
- rate of change [-30~30] except 0
- start : [-50~50]
  
| Term |1st | 2nd  |  3rd |  4th | 5th  |  6th | 7th  |  8th |
|---|---|---|---|---|---|---|---|---|
| value | -40 | -4  | 32  | 68  |   |   | | |





In [ ]:
def FillInTheSequence(difficulty=1 ,expr='latex'):
    header = ['Term',	'1st','2nd','3rd','4th','5th','6th','7th','8th']
    values = ['Value']
    numterms = len(header)-1

    if difficulty == 1: # easy
        if randint(0,1):
            start = getInt(-10, 10)
            ratio = getInt(2, 4)
            seq = GeoSeq(ratio, [1,start])
        else:
            start = getInt(-10, 10)
            diff = getInt(-4, 4)
            seq = ArithSeq(diff, [1,start])
    if difficulty == 2: # medium
        if randint(0,1):
            start = getInt(-10, 20)
            ratio = getInt(-4, 4, exclude=[-1,0,1])
            seq = GeoSeq(ratio, [1,start])
        else:
            start = getInt(-20, 20)
            diff = getInt(-15, 15)
            seq = ArithSeq(diff, [1,start])
    if difficulty == 3: # hard
        if randint(0,1):
            start = getInt(-20, 20)
            ratio = getInt(-6, 6, exclude=[-3,-2,-1,0,1,2,3])
            seq = GeoSeq(ratio, [1,start])
        else:
            start = getInt(-50, 50)
            diff = getInt(-30, 30)
            seq = ArithSeq(diff, [1,start])

    terms = latexify(seq.getTerms(numterms))
    for jj in range(1,numterms+1):
        if jj <= 4:
            values.append(signify(terms[jj]))
        else:
            values.append(r'\phantom{$'+ terms[jj] + '$}')
    problem = tableGenerator(header, [values], table_env=False)

    if expr == "latex":
        for jj in range(5,numterms+1):
            values[jj] = signify(terms[jj])
        answer = tableGenerator(header, [values], table_env=False)
    else:
        answer = terms

    return problem, answer
  

for i in range(3):
    problem, answer = FillInTheSequence(difficulty = randint(1,3))
    print(problem)
    print(answer)

NameError: ignored

### section2:
#### Use the given values in the table to determine a pattern and complete the table.

Easy
- case 1
- You	run	a	business	making	[birdhouses].	You	spend	[\$600]	to	start	your	business,	and	it	costs	you	[\$5.00] to	make	each	birdhouse.	

| number of birdthouse  |  1 | 2  |  3 |  4 | 5  |  6 |
|---|---|---|---|---|---|---|
| total cost to build  |   |   |   |   |   |   |

- case 2

Medium
- case 1

- case 2

Hard 
- case 1

- case 2








In [ ]:
def FillInTheTableWordProb(kind='lin', expr='latex'):
    problem, solns, labels = makeWordProb(kind, expr)
    
    if kind in ['lin','exp']:
        if type(solns[0]) != list:
            seq = ArithSeq(solns[0],solns[1]) if kind=='lin' else GeoSeq(solns[0],solns[1])
        else:
            seq = ArithSeq.fromPts(solns[0],solns[1]) if kind=='lin' else GeoSeq.fromPts(solns[0],solns[1])
        
        problem += seq.getTable(range(1,7), vals=False, vertical=False, labels=labels)
        
        answer = seq.getTable(range(1,7), vertical=False, labels=labels) if expr=='latex' else seq
    else:
        answer = 'Neither.'

    return problem, answer

for jj in range(10):
    problem, answer = FillInTheTableWordProb('exp')
    print(problem)
    print(answer)

The first year a toy manufacturer introduces a new toy, its sales total \$131,336.23. The company expects its sales to drop 63\% each succeeding year.{\renewcommand{\arraystretch}{1.5}	\begin{center}
		\begin{tabular}{|c|c|c|c|c|c|c|}
			\hline
			Year & $1$ & $2$ & $3$ & $4$ & $5$ & $6$ \\
			\hline
			\$ & \phantom{$131,336.23$} & \phantom{$48,594.41$} & \phantom{$17,979.93$} & \phantom{$6,652.57$} & \phantom{$2,461.45$} & \phantom{$910.74$} \\
			\hline
		\end{tabular}
	\end{center}}
{\renewcommand{\arraystretch}{1.5}	\begin{center}
		\begin{tabular}{|c|c|c|c|c|c|c|}
			\hline
			Year & $1$ & $2$ & $3$ & $4$ & $5$ & $6$ \\
			\hline
			\$ & $131,336.23$ & $48,594.41$ & $17,979.93$ & $6,652.57$ & $2,461.45$ & $910.74$ \\
			\hline
		\end{tabular}
	\end{center}}
A virus goes through a computer, infecting files. If 174 files were infected initially and the total number of files infected grows by a factor of seven every minute.{\renewcommand{\arraystretch}{1.5}	\begin{center}
		\begin{t

### Section 3:
#### instruction Evaluate	the	following	equations	when	x	=	{	1,	2,	3,	4,	5	}.		Organize	your	inputs	and	outputs	into	a	table	of	values	for	each	equation. Let	x	be	the	input	and	y	be	the	output.

- easy
- problem : $y = 4^x$

| x input | y output |
|---------|----------|
| 1       |          |
| 2       |          |
| 3       |          |
| 4       |          |
| 5       |          |

- answer

| x input | y output |
|---------|----------|
| 1       |      4    |
| 2       |      16    |
| 3       |      64    |
| 4       |      256    |
| 5       |       1028   |

- medium
- case 1
- problem $y = (-3)^x$

| x input | y output |
|---------|----------|
| 1       |          |
| 2       |          |
| 3       |          |
| 4       |          |
| 5       |          |

- answer

| x input | y output |
|---------|----------|
| 1       |     -3     |
| 2       |      9    |
| 3       |      -27    |
| 4       |       81   |
| 5       |     -243     |

- case 2
- problem $y = -3^x$

| x input | y output |
|---------|----------|
| 1       |          |
| 2       |          |
| 3       |          |
| 4       |          |
| 5       |          |

- answer

| x input | y output |
|---------|----------|
| 1       |    -3      |
| 2       |     -9     |
| 3       |     -27     |
| 4       |      -81    |
| 5       |       -243   |

- hard
- case 1
- problem $y = \pm2(-3)^x$

| x input | y output |
|---------|----------|
| 1       |          |
| 2       |          |
| 3       |          |
| 4       |          |
| 5       |          |

- answer

- case 2
- problem $y = \pm2 (3)^x$

| x input | y output |
|---------|----------|
| 1       |          |
| 2       |          |
| 3       |          |
| 4       |          |
| 5       |          |

- answer




In [ ]:
def FillInTheExpTable(difficulty=1, expr='latex'):
    labels = [r'$x$ input', r'$y$ output']

    if difficulty == 1: # easy
        ratio = getInt(2, 6)
        func = ExpFunc(ratio, 1)
    if difficulty == 2: # medium
        ratio = getInt(2, 6)
        if randint(0,1):
            ratio *= -1
            func = ExpFunc(ratio, 1)
        else:
            func = ExpFunc(ratio, -1)
    if difficulty == 3: # hard
        ratio = getInt(2, 6)
        start = getInt(-6, 6, exclude=[-1,0,1])
        if randint(0,1):
            ratio *= -1
        func = ExpFunc(ratio, start)

    problem = '$' + func.getBaseInt(notation='y') + r'$ \newline '
    problem += func.getTable(range(1,6),vals=False,labels=labels)
    if expr == "latex":
        answer = func.getTable(range(1,6),labels=labels)
    else:
        answer = func.subSet(range(1,6))

    return problem, answer
  

for i in range(6):
    problem, answer = FillInTheExpTable(difficulty = randint(1,3))
    print(problem)
    print(answer)

$y=-6\left(-2\right)^{x}$ \newline {\renewcommand{\arraystretch}{1.5}	\begin{center}
		\begin{tabular}{|c|c|}
			\hline
			$x$ input & $y$ output \\
			\hline
			$1$ & \phantom{$12$} \\
			\hline
			$2$ & \phantom{$-24$} \\
			\hline
			$3$ & \phantom{$48$} \\
			\hline
			$4$ & \phantom{$-96$} \\
			\hline
			$5$ & \phantom{$192$} \\
			\hline
		\end{tabular}
	\end{center}}
{\renewcommand{\arraystretch}{1.5}	\begin{center}
		\begin{tabular}{|c|c|}
			\hline
			$x$ input & $y$ output \\
			\hline
			$1$ & $12$ \\
			\hline
			$2$ & $-24$ \\
			\hline
			$3$ & $48$ \\
			\hline
			$4$ & $-96$ \\
			\hline
			$5$ & $192$ \\
			\hline
		\end{tabular}
	\end{center}}
$y=-3 \cdot 2^{x}$ \newline {\renewcommand{\arraystretch}{1.5}	\begin{center}
		\begin{tabular}{|c|c|}
			\hline
			$x$ input & $y$ output \\
			\hline
			$1$ & \phantom{$-6$} \\
			\hline
			$2$ & \phantom{$-12$} \\
			\hline
			$3$ & \phantom{$-24$} \\
			\hline
			$4$ & \phantom{$-48$} \\
			\hline
			$5$ & \phantom{$-96$} \

## 1.2.2  The	meaning	of	an	exponent


In [ ]:
def getExpandedForm(base, exp, start=1):
    """
    Return LaTeX formatted expanded exponential expression.
    
    Parameters
    ----------
        base : number (int, float, or Rational)
        exp : int
    """
    
    if base>0:
        factors = exp*[latex(base)]
    else:
        factors = exp*[r'\left(' + latex(base) + r'\right)']
    if start != 1:
        factors = [latex(start)] + factors
    return (r' \times ').join(factors)

def getExponentForm(base, exp, start=1):
    """Return LaTeX formatted exponential expression."""
    
    baseStr = latex(base)
    if type(base)!=int or base<0:
        baseStr = r'\left(' + baseStr + r'\right)'
    expStr = r'^{' + latex(exp) + r'}'
    start = latex(start)
    
    result = baseStr + expStr
    if start != '1':
        if type(base)!=int or base<0:
            result = start + result
        else:
            result = start + r'\cdot ' + result
    if base<0 and type(exp)==int: # Alternate solution for base<0; Does not handle case of start!=1
        baseStr = latex(abs(base))
        if type(base)!=int:
            baseStr = r'\left(' + baseStr + r'\right)'
        result += r' \text{ or } '
        alt = '-' + baseStr + expStr if exp%2==1 else baseStr + expStr
        result += alt

    return result

### Section 1:
#### instruction: Write	each	expression	using	an	exponent.	

intA = [1 ,10]
intB = [-10,-1]
intC = [-10,20]
intD = [-20,20]

- easy
  -  problem : intA $ \times $ intA $ \times $ intA $ \times $ intA $ \times $ intA 
  - answer : intA ^ 5
- medium
  - case 1
  -  problem : intB $ \times $ intB $ \times $ intB $ \times $ intB 
  - answer : intB ^ 4
  - case 2
  -  problem : intB $ \times $ intB $ \times $ intB $ \times $ intB $ \times $ intB
  - answer : -intB ^ 5
- hard # need to think about how to alternate signs $\pm$
  - case 1
  -  problem : $\frac{intC}{intD} \times \frac{intC}{intD} \times \frac{intC}{intD} \times \frac{intC}{intD} \times \frac{intC}{intD} $
  - answer : $\pm( \frac{intC}{intD})^ 5$
  - case 2
  -  problem : $\frac{intC}{intD} \times \frac{intC}{intD} \times \frac{intC}{intD} \times \frac{intC}{intD} \times \frac{intC}{intD} $
  - answer : $\pm( \frac{intC}{intD})^ 5$


In [ ]:
def ConvertToExponentForm(difficulty=1, expr="latex"):
    exp = getInt(4,9)
    
    if difficulty == 1: # easy
        base = getInt(1,10)
    if difficulty == 2: # medium
        base = getInt(-10,-1)
    if difficulty == 3: # hard
        base = sample([1,2,3,5,7,11,13],2)
        if randint(0,1):
            base[0] *= -1
        base = Rational(base[0],base[1]) if base[1]!=1 else Rational(base[1],base[0])
    
    problem = getExpandedForm(base, exp)
    if expr == 'latex':
        problem = '$' + problem + '$'
        answer = '$' + getExponentForm(base, exp) + '$'
    else:
        answer = {'base' : base, 'exp' : exp}

    return problem, answer

for jj in range(10):
    problem, answer = ConvertToExponentForm(difficulty=1)
    print(problem, r'\\')
    print(answer, r'\\')

$4 \times 4 \times 4 \times 4 \times 4 \times 4 \times 4 \times 4 \times 4$ \\
$\left(4\right)^{9}$ \\
$3 \times 3 \times 3 \times 3 \times 3 \times 3 \times 3 \times 3 \times 3$ \\
$\left(3\right)^{9}$ \\
$10 \times 10 \times 10 \times 10$ \\
$\left(10\right)^{4}$ \\
$10 \times 10 \times 10 \times 10 \times 10 \times 10$ \\
$\left(10\right)^{6}$ \\
$5 \times 5 \times 5 \times 5$ \\
$\left(5\right)^{4}$ \\
$9 \times 9 \times 9 \times 9 \times 9$ \\
$\left(9\right)^{5}$ \\
$1 \times 1 \times 1 \times 1$ \\
$\left(1\right)^{4}$ \\
$5 \times 5 \times 5 \times 5 \times 5 \times 5 \times 5 \times 5$ \\
$\left(5\right)^{8}$ \\
$4 \times 4 \times 4 \times 4 \times 4 \times 4 \times 4 \times 4$ \\
$\left(4\right)^{8}$ \\
$1 \times 1 \times 1 \times 1 \times 1$ \\
$\left(1\right)^{5}$ \\



### Section 2:
#### instruction: A)	Write	each	expression	in	expanded	form. B) Then	calculate	the	value	of	the	expression

---
- intA = [2 , 10]
- intB = [2 , 10]
- intC = [2 , 10]
- intD = [-10 , 20]
- intE = [-20 , 20]
---

- easy
  - $intA^{intB}$
  - $5^{3}$
  - answer : a) 
expanded form: $5 \times 5 \times 5$
  - answer : b) 
value of the expression: $125$
- medium
  - $intD * intA^{intB}$
  - $4 * 5^{3}$
  - answer : a) $4 \times 5 \times 5 \times 5$
  - answer : b) $600$
- hard
  - case 1
  - $intD *(\frac{intA}{intB})^{intC}$
  - $-8 (\frac{1}{3})^{3}$
  - answer : a) $-8 \times \frac{1}{3} \times \frac{1}{3} \times \frac{1}{3}$
  - answer : b) $-\frac{8}{27}$



In [ ]:
def ConvertToExpandedForm(difficulty=1, expr="latex"):
    exp = getInt(2,10)
    
    if difficulty == 1: # easy
        base = getInt(2,10)
        start = 1
        value = base**exp
    if difficulty == 2: # medium
        base = getInt(2,10)
        start = getInt(-10,20,exclude=[-1,0,1])
        value = start*base**exp
    if difficulty == 3: # hard
        base = sample([1,2,3,5,7],2)
        base = Rational(base[0],base[1]) if base[1]!=1 else Rational(base[1],base[0])
        start = getInt(-10,20,exclude=[-1,0,1])
        value = simplify(start*base**exp)
    
    problem = getExponentForm(base, exp, start)
    if expr == 'latex':
        problem = '$' + problem + '$'
        answer = '$' + getExpandedForm(base, exp, start) + r'$ \newline '
        answer += signify(latex(value))
    else:
        answer = {'base' : base, 'exp' : exp, 'start' : start, 'value' : value}

    return problem, answer

for jj in range(10):
    problem, answer = ConvertToExpandedForm(difficulty=3)
    print(problem, r'\\')
    print(answer, r'\\')

$7\left(\frac{2}{5}\right)^{3}$ \\
$7 \times \frac{2}{5} \times \frac{2}{5} \times \frac{2}{5}$ \newline $\frac{56}{125}$ \\
$8\left(\frac{1}{3}\right)^{8}$ \\
$8 \times \frac{1}{3} \times \frac{1}{3} \times \frac{1}{3} \times \frac{1}{3} \times \frac{1}{3} \times \frac{1}{3} \times \frac{1}{3} \times \frac{1}{3}$ \newline $\frac{8}{6561}$ \\
$10\left(\frac{7}{2}\right)^{10}$ \\
$10 \times \frac{7}{2} \times \frac{7}{2} \times \frac{7}{2} \times \frac{7}{2} \times \frac{7}{2} \times \frac{7}{2} \times \frac{7}{2} \times \frac{7}{2} \times \frac{7}{2} \times \frac{7}{2}$ \newline $\frac{1412376245}{512}$ \\
$10\left(\frac{5}{2}\right)^{4}$ \\
$10 \times \frac{5}{2} \times \frac{5}{2} \times \frac{5}{2} \times \frac{5}{2}$ \newline $\frac{3125}{8}$ \\
$10\left(\frac{2}{3}\right)^{9}$ \\
$10 \times \frac{2}{3} \times \frac{2}{3} \times \frac{2}{3} \times \frac{2}{3} \times \frac{2}{3} \times \frac{2}{3} \times \frac{2}{3} \times \frac{2}{3} \times \frac{2}{3}$ \newline $\frac{5120}{19683}

## 1.2.3 Finding Patterns In Geometric Shapes

Growing, growing Dots

In [ ]:
# # Latex setup code

# \documentclass{article}
# \usepackage{tikz}

# \usetikzlibrary{math}

# \usepackage{environ}
# \makeatletter
# \newsavebox{\measure@tikzpicture}
# \NewEnviron{scaletikzpicturetowidth}[1]{%
#   \def\tikz@width{#1}%
#   \def\tikzscale{1}\begin{lrbox}{\measure@tikzpicture}%
#   \BODY
#   \end{lrbox}%
#   \pgfmathparse{#1/\wd\measure@tikzpicture}%
#   \edef\tikzscale{\pgfmathresult}%
#   \BODY
# }
# \makeatother



# % arg1 = diff
# % arg2 = zero term
# % arg3 = start fig
# % arg4 = end fig
# % arg5 = fig spacing
# \newcommand{\dotPattern}[5]
# {\begin{scaletikzpicturetowidth}{0.5\textwidth}
# \begin{tikzpicture}[scale=\tikzscale]
#     \foreach \step in {#3,...,#4}
#     {
#         \tikzmath{
#         \xmax = #1*\step + #2;
#         \y = (#4 - \step)*#5;
#         }
#         \foreach \x in {1,...,\xmax}
#             \fill[red!50] (\x,\y) ellipse (5pt and 4pt);
#     }
# \end{tikzpicture}
# \end{scaletikzpicturetowidth}
# }

# % arg1 = arm growth (1/2 of diff)
# % arg2 = zero term
# % arg3 = start fig
# % arg4 = end fig
# % arg5 = fig spacing
# \newcommand{\wideLdotPattern}[5]
# {\begin{scaletikzpicturetowidth}{0.5\textwidth}
# \begin{tikzpicture}[scale=\tikzscale]
#     \foreach \step in {#3,...,#4}
#     {
#         \tikzmath{
#         \arm = #2 - 1 + #1*\step;
#         \xstart = 1 + (\step-1)*(#2 + #5 + #1*\step/2);
#         }
#         \foreach \block in {0,...,\arm}
#         {
#             \tikzmath{\x = \xstart + \block;}
#             \fill[red!50] (\x,1) ellipse (5pt and 4pt);
#         }
#         \tikzmath{\ymax = #1*\step + 1;}
#         \foreach \y in {2,...,\ymax}
#             \fill[red!50] (\xstart,\y) ellipse (5pt and 4pt);
#     }
# \end{tikzpicture}
# \end{scaletikzpicturetowidth}
# }

# % arg1 = arm growth (1/3 of diff)
# % arg2 = zero term
# % arg3 = start fig
# % arg4 = end fig
# % arg5 = fig spacing
# \newcommand{\tallLdotPattern}[5]
# {\begin{scaletikzpicturetowidth}{0.5\textwidth}
# \begin{tikzpicture}[scale=\tikzscale]
#     \foreach \step in {#3,...,#4}
#     {
#         \tikzmath{
#         \arm = #1*\step;
#         \xstart = \step + (\step-1)*(#5 + #1*\step/2);
#         }
#         \foreach \block in {0,...,\arm}
#         {
#             \tikzmath{\x = \xstart + \block;}
#             \fill[red!50] (\x,1) ellipse (5pt and 4pt);
#         }
#         \tikzmath{\ymax = \arm + #2;}
#         \foreach \y in {2,...,\ymax}
#             \fill[red!50] (\xstart,\y) ellipse (5pt and 4pt);
#     }
# \end{tikzpicture}
# \end{scaletikzpicturetowidth}
# }

# % arg1 = arm growth (1/3 of diff)
# % arg2 = zero term (must be odd)
# % arg3 = start fig
# % arg4 = end fig
# % arg5 = fig spacing
# \newcommand{\wideTdotPattern}[5]
# {\begin{scaletikzpicturetowidth}{0.5\textwidth}
# \begin{tikzpicture}[scale=\tikzscale]
#     \tikzmath{\halfFirst = (#2-1)/2;}
#     \foreach \step in {#3,...,#4}
#     {
#         \tikzmath{
#         \arm = #1*\step + \halfFirst;
#         \xmid = \halfFirst + #1*\step^2 + #5*\step + (\step-1)*#2;
#         \xstart = \xmid - \arm;
#         \basesteps = 2*\arm;
#         }
#         \foreach \block in {0,...,\basesteps}
#         {
#             \tikzmath{\x = \xstart + \block;}
#             \fill[red!50] (\x,1) ellipse (5pt and 4pt);
#         }
#         \tikzmath{\ymax = \arm - \halfFirst + 1;}
#         \foreach \y in {2,...,\ymax}
#             \fill[red!50] (\xmid,\y) ellipse (5pt and 4pt);
#     }
# \end{tikzpicture}
# \end{scaletikzpicturetowidth}
# }

# % arg1 = arm growth (1/3 of diff)
# % arg2 = zero term
# % arg3 = start fig
# % arg4 = end fig
# % arg5 = fig spacing
# \newcommand{\tallTdotPattern}[5]
# {\begin{scaletikzpicturetowidth}{0.5\textwidth}
# \begin{tikzpicture}[scale=\tikzscale]
#     \foreach \step in {#3,...,#4}
#     {
#         \tikzmath{
#         \arm = #1*\step;
#         \xmid = #1*\step^2 + \step + (\step-1)*#5;
#         \xstart = \xmid - \arm;
#         \basesteps = 2*\arm;
#         }
#         \foreach \block in {0,...,\basesteps}
#         {
#             \tikzmath{\x = \xstart + \block;}
#             \fill[red!50] (\x,1) ellipse (5pt and 4pt);
#         }
#         \tikzmath{\ymax = \arm + #2;}
#         \foreach \y in {2,...,\ymax}
#             \fill[red!50] (\xmid,\y) ellipse (5pt and 4pt);
#     }
# \end{tikzpicture}
# \end{scaletikzpicturetowidth}
# }

# % arg1 = width growth
# % arg2 = length growth
# % arg3 = zero term
# % arg4 = start fig
# % arg5 = end fig
# % arg6 = fig spacing
# \newcommand{\polyRectanglePattern}[6]
# {\begin{scaletikzpicturetowidth}{0.5\textwidth}
# \begin{tikzpicture}[scale=\tikzscale]
#     \foreach \step in {#4,...,#5}
#     {
#         \tikzmath{
#         \length = #2*\step + 1;
#         \width = #1*\step + #3 - 1;
#         \xstart = 1 + (\step-1)*(#3 + #6 + 1 + #1*\step/2);
#         }
#         \foreach \block in {0,...,\width}
#         {
#             \tikzmath{\x = \xstart + \block;}
#             \foreach \y in {1,...,\length}
#                 \fill[red!50] (\x,\y) ellipse (5pt and 4pt);
#         }
#     }
# \end{tikzpicture}
# \end{scaletikzpicturetowidth}
# }

# % arg1 = length factor
# % arg2 = width factor
# % arg3 = zero term
# % arg4 = start fig
# % arg5 = end fig
# % arg6 = fig spacing
# \newcommand{\expRectanglePattern}[6]
# {\begin{scaletikzpicturetowidth}{0.5\textwidth}
# \begin{tikzpicture}[scale=\tikzscale]
#     \tikzmath{\width = #3 - 1;}
#     \foreach \step in {#4,...,#5}
#     {
#         \tikzmath{\length = #1^\step;}
#         \ifnum #2=1
#             \tikzmath{\xstart = 1 + (\step-1)*(\width + #6);}
#         \else
#             \tikzmath{
#             \width = #3*#2^\step - 1;
#             \xstart = 1 + #6*(\step-1) + (#3*#2 - \width + 1)/(1-#2);
#             }
#         \fi
#         \foreach \block in {0,...,\width}
#         {
#             \tikzmath{\x = \xstart + \block;}
#             \foreach \y in {1,...,\length}
#                 \fill[red!50] (\x,\y) ellipse (5pt and 4pt);
#         }
#     }
# \end{tikzpicture}
# \end{scaletikzpicturetowidth}
# }












# \begin{document}


# \noindent Example 1
# \\

# \dotPattern{2}{1}{1}{4}{1}

# \vspace{2cm}


# \noindent Example 2
# \\

# \wideTdotPattern{2}{3}{1}{3}{2}

# \vspace{2cm}


# \noindent Example 3
# \\

# \tallTdotPattern{2}{3}{1}{3}{2}

# \vspace{2cm}


# \noindent Example 4
# \\

# \wideLdotPattern{2}{3}{1}{3}{2}

# \vspace{2cm}


# \noindent Example 5
# \\

# \tallLdotPattern{2}{3}{1}{3}{2}

# \vspace{2cm}


# \noindent Example 6
# \\

# %linear
# \polyRectanglePattern{0}{1}{3}{1}{3}{2}

# \vspace{2cm}


# \noindent Example 8
# \\

# %doubling
# \expRectanglePattern{2}{1}{5}{1}{3}{2}

# \vspace{2cm}


# \noindent Example 9 
# \\

# %quadrupling
# \expRectanglePattern{2}{2}{2}{1}{3}{2}



# \end{document}

### Section 1:


Draw Step 4 and Step 5.



In [ ]:
def extendThePattern(kind='lin'):
    if kind == 'lin':
        case = randint(1,6)
        growth, intercept = [randint(1,4),randint(-1,3)]
        if case == 1: # line of dots
            diff = growth
            problem, answer = [r'\dotPattern']*2
        elif case in [2,3]: # wide  or tall L
            diff = 2*growth
            problem, answer = [r'\wideLdotPattern']*2 if case==2 else [r'\tallLdotPattern']*2
        elif case in [4,5]: # wide or tall T
            diff = 3*growth
            problem, answer = [r'\wideTdotPattern']*2 if case==4 else [r'\tallTdotPattern']*2
        elif case == 6: # rectangle
            diff = intercept*growth
            problem, answer = [r'\polyRectanglePattern{0}']*2

        seq = ArithSeq(diff, intercept)
        problem += brackify(growth) + brackify(intercept) + '{1}{3}{2}'
        answer += brackify(growth) + brackify(intercept) + '{4}{5}{2}'
    elif kind == 'exp':
        growth1, growth2 = randint(2,3), randint(1,2)
        seq = GeoSeq(growth1*growth2, randint(2,3))
        problem, answer = [r'\expRectanglePattern']*2
        problem += brackify(growth1) + brackify(growth2) + brackify(seq.intercept) + '{1}{3}{2}'
        answer += brackify(growth1) + brackify(growth2) + brackify(seq.intercept) + '{4}{5}{2}'

    return problem, answer

for jj in range(10):
    problem, answer = extendThePattern()
    print(problem)
    print(answer)

\dotPattern{2}{2}{1}{3}{2}
\dotPattern{2}{2}{4}{5}{2}
\dotPattern{4}{2}{1}{3}{2}
\dotPattern{4}{2}{4}{5}{2}
\tallLdotPattern{3}{-1}{1}{3}{2}
\tallLdotPattern{3}{-1}{4}{5}{2}
\wideLdotPattern{1}{0}{1}{3}{2}
\wideLdotPattern{1}{0}{4}{5}{2}
\dotPattern{1}{0}{1}{3}{2}
\dotPattern{1}{0}{4}{5}{2}
\polyRectanglePattern{0}{2}{2}{1}{3}{2}
\polyRectanglePattern{0}{2}{2}{4}{5}{2}
\tallLdotPattern{4}{1}{1}{3}{2}
\tallLdotPattern{4}{1}{4}{5}{2}
\dotPattern{4}{-1}{1}{3}{2}
\dotPattern{4}{-1}{4}{5}{2}
\tallTdotPattern{2}{0}{1}{3}{2}
\tallTdotPattern{2}{0}{4}{5}{2}
\wideLdotPattern{2}{0}{1}{3}{2}
\wideLdotPattern{2}{0}{4}{5}{2}


## 1.3.1 Reading The Table




### Section 1
#### instruction : Use the tables to find the indicated values.

easy 

| n | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 |
| :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- |
| f(n) | -8 | -3 | 2 | 7 | 12 | 17 | 22 | 27 | 32 | 37 |

- problem 
- a) Find $f(5)$
- b) Find $f(9)$
- c) Find $n$ that goes with $f(n) = 22$.
- d) Find $n$ that goes with $f(n) = -3$.

medium 

| n | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 |
| :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- |
| f(n) | -8 | -3 | 2 | 7 | 12 | 17 | 22 | 27 | 32 | 37 |

- problem 
- a) When $f(n) = 22$, what is the value of $n$?
- b) What is the value of $f(n-1)$?
- c) When $f(n) = -3$, what is the value of $n$?
- d) What is the value of $f(n+1)$?

hard 

| n | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 |
| :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- |
| f(n) | -8 | -3 | 2 | 7 | 12 | 17 | 22 | 27 | 32 | 37 |

- problem 
- a) When $f(n) = 22$, what is the value of $n$?
- b) What is the value of $f(n-6)$?
- c) When $f(n) = -3$, what is the value of $n$?
- d) What is the value of $f(n+3)$?


In [ ]:
def ReadTheFuncTable(difficulty=1, expr='latex'):
    header = [r'$n$'] + signify(list(range(1,11)))
    outputs =  [r'f(n)']

    if difficulty == 1: # easy
        outputs += random.sample(list(range(-10,31)), 10)
    elif difficulty == 2: # medium
        outputs += random.sample(list(range(-30,51)), 10)
    elif difficulty == 3: # hard
        outputs += random.sample(list(range(-100,301)), 10)

    problem = tableGenerator(header, [signify(outputs)])

    if difficulty == 1: # easy
        choices = random.sample(list(range(1,11)), 2)           # random inputs
        choices += random.sample(outputs[1:], 2)                # random outputs
        solutions = [outputs[choices[0]], outputs[choices[1]]]
        solutions += [outputs.index(choices[2]), outputs.index(choices[3])]

        parta = fr'(a) Find $f({choices[0]})$. \newline '
        partb = fr'(b) Find $f({choices[1]})$. \newline '
        partc = fr'(c) Find $n$ such that $f(n) = {choices[2]}$. \newline '
        partd = fr'(d) Find $n$ such that $f(n) = {choices[3]}$. \newline '
        answer = fr'(a) $f({choices[0]}) = {solutions[0]}$ \newline '
        answer += fr'(b) $f({choices[1]}) = {solutions[1]}$ \newline '
        answer += fr'(c) $n = {solutions[2]}$. \newline (d) $n = {solutions[3]}$. \newline '
    elif difficulty >= 2: # medium and hard
        choices = random.sample(outputs[2:-1], 2)               # random outputs
        solutions = [outputs.index(choices[0]), outputs.index(choices[1])]
        if difficulty == 2:
            shifts = [1,1]
        elif difficulty == 3:    # random shifts
            shifts = [random.randint(1,solutions[0]-1), random.randint(1,10-solutions[1])]
        solutions.insert(1,outputs[solutions[0]-shifts[0]])      # put solutions in same order as problem
        solutions.append(outputs[solutions[2]+shifts[1]])

        parta = fr'(a) When $f(n) = {choices[0]}$, what is the value of $n$? \newline '
        partb = fr'(b) What is the value of $f(n-{shifts[0]})$? \newline '
        partc = fr'(c) When $f(n) = {choices[1]}$, what is the value of $n$? \newline '
        partd = fr'(d) What is the value of $f(n+{shifts[1]})$? \newline '
        answer = fr'(a) $n = {solutions[0]}$ \newline '
        answer += fr'(b) $f(n-{shifts[0]}) = f({solutions[0]}-{shifts[0]}) = f({solutions[0]-shifts[0]}) = {solutions[1]}$ \newline '
        answer += fr'(c) $n = {solutions[2]}$ \newline '
        answer += fr'(d) $f(n+{shifts[1]}) = f({solutions[2]}+{shifts[1]}) = f({solutions[2]+shifts[1]}) = {solutions[3]}$ \newline'
    
    problem += parta + partb + partc + partd

    return problem, answer
  

for i in range(4):
    problem, answer = ReadTheFuncTable(3)
    print(problem)
    print(answer)
    print(r'ss')

{\renewcommand{\arraystretch}{1.5}	\begin{center}
		\begin{tabular}{|c|c|c|c|c|c|c|c|c|c|c|}
			\hline
			$n$ & $1$ & $2$ & $3$ & $4$ & $5$ & $6$ & $7$ & $8$ & $9$ & $10$ \\
			\hline
			$f(n)$ & $210$ & $56$ & $-54$ & $121$ & $175$ & $2$ & $233$ & $-48$ & $71$ & $-51$ \\
			\hline
		\end{tabular}
	\end{center}}(a) When $f(n) = 56$, what is the value of $n$? \newline (b) What is the value of $f(n-1)$? \newline (c) When $f(n) = 121$, what is the value of $n$? \newline (d) What is the value of $f(n+3)$? \newline 
(a) $n = 2$ \newline (b) $f(n-1) = f(2-1) = f(1) = 210$ \newline (c) $n = 4$ \newline (d) $f(n+3) = f(4+3) = f(7) = 233$ \newline
ss
{\renewcommand{\arraystretch}{1.5}	\begin{center}
		\begin{tabular}{|c|c|c|c|c|c|c|c|c|c|c|}
			\hline
			$n$ & $1$ & $2$ & $3$ & $4$ & $5$ & $6$ & $7$ & $8$ & $9$ & $10$ \\
			\hline
			$f(n)$ & $97$ & $33$ & $19$ & $124$ & $94$ & $123$ & $77$ & $83$ & $-69$ & $253$ \\
			\hline
		\end{tabular}
	\end{center}}(a) When $f(n) = 33$, what is the value

## 1.3.2 Arithmetic Sequences

In [ ]:
def getSeqAnswer(seq, prob='common', blanks=False, expr="latex", numTerms=6):
    if expr=="latex":
        answer = ''
        if prob in ['common','all'] :
            answer += '$d = ' if type(seq)==ArithSeq else '$r = '
            answer += latexify(seq.common, seq.precision) + '$'
        if prob in ['explicit','all']:
            if len(answer)>0:
                answer += r' \newline '
            answer += '$' + seq.getExplicit() + r' \quad\text{or}\quad ' + seq.getExplicit(0) + '$'
        if prob in ['recursive','all']:
            if len(answer)>0:
                answer += r' \newline '
            answer += '$' + seq.getRecursive() + r' \quad\text{or}\quad ' + seq.getRecursive(0) + '$'
        
        if blanks:
            if len(answer)>0:
                answer += r' \newline '
            answer += signify(seq.getSeqStr(range(1,numTerms+1))) 
    else:
        seq.getTerms(numTerms)
        answer = seq

    return answer

def Arithmetic_Sequence_Problem(difficulty=1, prob='common', blanks=False, expr="latex"):
    if difficulty == 1: # easy
        if randint(0,1):
            arith = ArithSeq(randint(1,10), [1,randint(1,10)])
        else:
            arith = ArithSeq(randint(-10,-1), [1,randint(-10,-1)])
    
    elif difficulty == 2: # medium (TODO)
        if randint(0,1):
            arith = ArithSeq(Rational(randint(1,5),randint(2,5)), [1,randint(-5,5)])
        else:
            arith = ArithSeq(Rational(randint(-5,-1),randint(2,5)), [1,randint(-5,5)])

    elif difficulty == 3: # hard (TODO)
        if randint(0,1):
            arith = ArithSeq(Rational(randint(1,7),randint(2,7)), [1,Rational(randint(1,5),randint(2,5))])
        else:
            arith = ArithSeq(Rational(getInt(-7,7),randint(2,7)), [1,Rational(randint(-5,-1),randint(2,5))])
    
    nums = [1,2,'','','',''] if blanks else [1,2,3,4,5]
    problem = signify(arith.getSeqStr(nums))
    answer = getSeqAnswer(arith, prob, blanks, expr)
            
    return problem, answer

### Section 1
#### instruction : Identify the constant difference. 

---
- startA = [-10,10]
- startB = [-20,20]
- floatA = [ 0.3 , 0.5 , 0.9]
- floatB = [ -9.9,9.9]
- intA = [-4,-3,-2,-1,1,2,3,4]
- intB = [-4,-3,-2,-1,1,2,3,4]
- intC = [-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,1,2,3,4,5,6,7,8,9,10]
- intD = [-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,1,2,3,4,5,6,7,8,9,10]
- intC = 
- intD = 
---

- easy
  - case 1
  - problem : What is the the common difference?
  $$ \text{Sequence}: 3,9,15,21,27$$
  - answer : $ d = 6$
  - case 2
  - problem : What is the the common difference?
  $$ \text{Sequence}: 3,1,-1,-3,-5$$
  - answer: $ d = -2$
- medium
  - case 1
  - problem : What is the the common difference?
  $$ \text{Sequence}: \frac{1}{2},1,\frac{3}{2},2,\frac{5}{2}$$
  - answer : $ d = \frac{1}{2}$
  - case 2
  - problem : What is the the common difference?
  $$ \text{Sequence}: -\frac{1}{2},-\frac{5}{6},-\frac{7}{6},-\frac{3}{2},-\frac{11}{6}$$
  - answer : $ d = -\frac{1}{3}$
- hard
  - case 1
  - problem : What is the the common difference?
  $$ \text{Sequence}: 3\frac{1}{2},4,4\frac{1}{2},5,5\frac{1}{2}$$
  -  answer : $ d = \frac{1}{2}$
  - case 2
  - problem : What is the the common difference?
  $$ \text{Sequence}: 3\frac{1}{3},1,-1\frac{1}{3},-3\frac{2}{3},-6$$
  - answer : $ d = -\frac{7}{3}$

In [ ]:
def getArith(start,diff,num):
  compiledlist=[]
  curr_term=start
  for i in range(1,num+1):
    compiledlist.append(curr_term)
    curr_term = curr_term + diff
  return compiledlist

def Arithmatic_Sequence_Diff(difficulty=1, expr="latex"):
  choicelist1=[1,2,3,4,5,6,7,8,9,10]
  choicelist2=[-10,-9,-8,-7,-6,-5,-4,-3,-2,-1]
  if difficulty == 1:
    case=randint(1,2)
    if case == 1:
      rand1=randint(1,10)
      rand2=choice(choicelist1)
      arith=[]
      arith = getArith(rand1,rand2,5)
    if case == 2:
      rand1=randint(-10,-1)
      rand2=choice(choicelist2)
      arith=[]
      arith = getArith(rand1,rand2,5)
    problem = r'$\text{Sequence:\hspace{1mm}} '+fr'{arith[0]} , {arith[1]}, {arith[2]} , {arith[3]}, {arith[4]}$'
    answer = '$'+str(rand2)+'$'
    return problem,answer
    
  elif difficulty == 2:
    case=randint(1,2)
    if case == 1:
      numerator1 = randint(1,5)
      denominator1 = randint(2,5)
      numerator2 = randint(1,5)
      denominator2 = randint(2,5)
      start= Rational(numerator1,denominator1)
      diff= Rational(numerator2,denominator2)
      arith=[]
      arith.append(sympy.simplify(start))
      for i in range(4):
        arith.append(sympy.simplify(arith[i] + diff))
    if case == 2:
      numerator1 = randint(1,5)
      denominator1 = randint(2,5)
      numerator2 = randint(-5,-1)
      denominator2 = randint(2,5)
      start= Rational(numerator1,denominator1)
      diff= Rational(numerator2,denominator2)
      arith=[]
      arith.append(sympy.simplify(start))
      for i in range(4):
        arith.append(sympy.simplify(arith[i] + diff))
    
    if expr == 'latex':
      latexify=[]
      for iv in range(len(arith)):
        latexify.append(sympy.latex(arith[iv]))
      problem = r'$\text{Sequence:\hspace{1mm}} '+fr'{latexify[0]} , {latexify[1]}, {latexify[2]} , {latexify[3]}, {latexify[4]}$'
      answer = '$'+sympy.latex(sympy.simplify(diff))+'$'
      return problem,answer
    else:
      problem = r'\text{Sequence:\hspace{1mm}} '+fr'{arith[0]} , {arith[1]}, {arith[2]} , {arith[3]}, {arith[4]}'
      answer = sympy.simplify(diff)
      return problem,answer

  elif difficulty == 3:
    case=randint(1,2)
    if case == 1:
      front = randint(2,5)
      numerator1 = randint(1,7)
      denominator1 = randint(2,7)
      numerator2 = randint(1,5)
      denominator2 = randint(2,5)
      start= front*Rational(numerator1,denominator1)
      diff= Rational(numerator2,denominator2)
      arith=[]
      arith.append(sympy.simplify(start))
      for i in range(4):
        arith.append(sympy.simplify(arith[i] + diff))
    if case == 2:
      front = randint(2,5)
      numerator1 = randint(1,7)
      denominator1 = randint(2,7)
      numerator2 = randint(-5,-1)
      denominator2 = randint(2,5)
      start= front * Rational(numerator1,denominator1)
      diff= Rational(numerator2,denominator2)
      arith=[]
      arith.append(sympy.simplify(start))
      for i in range(4):
        arith.append(sympy.simplify(arith[i] + diff))

    if expr == 'latex':
      latexify=[]
      for iv in range(len(arith)):
        num , dem = arith[iv].as_numer_denom()
        front= num//dem
        top = num%dem
        bot = dem
        if top == 0:
          lat= str(front)
        elif front == 0:
          lat= r'\frac{'+str(top)+'}{'+str(bot)+'}'
        else:
          lat= str(front)+r'\frac{'+str(top)+'}{'+str(bot)+'}'
        latexify.append(lat)
      problem = r'$\text{Sequence:\hspace{1mm}} '+fr'{latexify[0]} , {latexify[1]}, {latexify[2]} , {latexify[3]}, {latexify[4]}$'
      answer = '$'+sympy.latex(sympy.simplify(diff))+'$'
      return problem,answer
    else:
      problem = r'\text{Sequence:\hspace{1mm}} '+fr'{arith[0]} , {arith[1]}, {arith[2]} , {arith[3]}, {arith[4]}'
      answer = sympy.simplify(diff)
      return problem,answer

for i in range(5):
  a , b = Arithmatic_Sequence_Diff(1,"latex")
  print(a," common diff: ",b)
'''
for i in range(5):
  a , b = Arithmatic_Sequence_Diff(2,"latex")
  print(a," common diff: ",b)
for i in range(50):
  a , b = Arithmatic_Sequence_Diff(3,"latex")
  print(a," common diff: ",b,r"\newline")
'''

$\text{Sequence:} 8 , 12, 16 , 20, 24$  common diff:  $4$
$\text{Sequence:} 8 , 11, 14 , 17, 20$  common diff:  $3$
$\text{Sequence:} -1 , -9, -17 , -25, -33$  common diff:  $-8$
$\text{Sequence:} -1 , -7, -13 , -19, -25$  common diff:  $-6$
$\text{Sequence:} -2 , -5, -8 , -11, -14$  common diff:  $-3$


'\nfor i in range(5):\n  a , b = Arithmatic_Sequence_Diff(2,"latex")\n  print(a," common diff: ",b)\nfor i in range(50):\n  a , b = Arithmatic_Sequence_Diff(3,"latex")\n  print(a," common diff: ",b,r"\newline")\n'

### Section 2
#### instruction : Identify the recursive equation. 

- easy
  - case 1
  - problem : What is the the common difference? $$ \text{Sequence}: 3,9,15,21,27$$
  - answer : $ f(n)=f(n-1)+ 6$ , $f(1) = 3$
  - case 2
  - problem : What is the the common difference? $$ \text{Sequence}: 3,1,-1,-3,-5$$
  - answer: $f(n)=f(n-1) - 2$ , $ f(1) = 3 $
- medium
  - case 1
  - problem : What is the the common difference?
  $$ \text{Sequence}: \frac{1}{2},1,\frac{3}{2},2,\frac{5}{2}$$
  - answer : $f(n)=f(n-1)+\frac{1}{2}$ , $ f(1) = \frac{1}{2}$
  - case 2
  - problem : What is the the common difference?
  $$ \text{Sequence}: -\frac{1}{2},-\frac{5}{6},-\frac{7}{6},-\frac{3}{2},-\frac{11}{6}$$
  - answer : $f(n)=f(n-1) -\frac{1}{3}$ , $-\frac{1}{2}$
- hard
  - case 1
  - problem : What is the the common difference?
  $$ \text{Sequence}: 3\frac{1}{2},4,4\frac{1}{2},5,5\frac{1}{2}$$
  -  answer : $f(n)=f(n-1) -\frac{1}{2}$ , $3\frac{1}{2}$
  - case 2
  - problem : What is the the common difference?
  $$ \text{Sequence}: 3\frac{1}{3},1,-1\frac{1}{3},-3\frac{2}{3},-6$$
  - answer : $ f(n)=f(n-1) -\frac{7}{3}$ , $ 3\frac{1}{3}$

In [ ]:
import sympy

def Arithmetic_Sequence_Recursive(difficulty=1, expr="latex"):
  choicelist1=[1,2,3,4,5,6,7,8,9,10]
  choicelist2=[-10,-9,-8,-7,-6,-5,-4,-3,-2,-1]
  if difficulty == 1:
    case=randint(1,2)
    if case == 1:
      rand1=randint(1,10)
      rand2=choice(choicelist1)
      arith=[]
      arith = getArith(rand1,rand2,5)
    if case == 2:
      rand1=randint(-10,-1)
      rand2=choice(choicelist2)
      arith=[]
      arith = getArith(rand1,rand2,5)
    if expr == 'latex':
      problem = r'$\text{Sequence:} '+fr'{arith[0]} , {arith[1]}, {arith[2]} , {arith[3]}, {arith[4]}$'
      if rand2 > 0:
        answer = '$'+r'f(n)=f(n-1)+'+str(rand2)+'$, $f(1)='+f'{arith[0]}'+'$'
      else:
        answer = '$'+r'f(n)=f(n-1)'+str(rand2)+'$, $f(1)='+f'{arith[0]}'+'$'
    else:
      problem = r'\text{Sequence:} '+fr'{arith[0]} , {arith[1]}, {arith[2]} , {arith[3]}, {arith[4]}'
      if diff > 0:
        answer = r'f(n)=f(n-1)+'+str(rand2)+', f(1)='+f'{arith[0]}'
      else:
        answer = r'f(n)=f(n-1)'+str(rand2)+', f(1)='+f'{arith[0]}'
    return problem,answer
    
  elif difficulty == 2:
    case=randint(1,2)
    if case == 1:
      numerator1 = randint(1,5)
      denominator1 = randint(2,5)
      numerator2 = randint(1,5)
      denominator2 = randint(2,5)
      start= Rational(numerator1,denominator1)
      diff= Rational(numerator2,denominator2)
      arith=[]
      arith.append(sympy.simplify(start))
      for i in range(4):
        arith.append(sympy.simplify(arith[i] + diff))
    if case == 2:
      numerator1 = randint(1,5)
      denominator1 = randint(2,5)
      numerator2 = randint(-5,-1)
      denominator2 = randint(2,5)
      start= Rational(numerator1,denominator1)
      diff= Rational(numerator2,denominator2)
      arith=[]
      arith.append(sympy.simplify(start))
      for i in range(4):
        arith.append(sympy.simplify(arith[i] + diff))
    
    if expr == 'latex':
      latexify=[]
      for iv in range(len(arith)):
        latexify.append(sympy.latex(arith[iv]))
      problem = r'$\text{Sequence:} '+fr'{latexify[0]} , {latexify[1]}, {latexify[2]} , {latexify[3]}, {latexify[4]}$'
      if diff > 0:
        answer = '$'+r'f(n)=f(n-1)+'+sympy.latex(sympy.simplify(diff))+'$, $f(1)='+f'{arith[0]}'+'$'
      else:
        answer = '$'+r'f(n)=f(n-1)'+sympy.latex(sympy.simplify(diff))+'$, $f(1)='+f'{arith[0]}'+'$'
    else:
      problem = r'\text{Sequence:} '+fr'{arith[0]} , {arith[1]}, {arith[2]} , {arith[3]}, {arith[4]}'
      if diff > 0:
        answer = r'f(n)=f(n-1)+'+str(sympy.simplify(diff))+', f(1)='+f'{arith[0]}'
      else:
        answer = r'f(n)=f(n-1)'+str(sympy.simplify(diff))+', f(1)='+f'{arith[0]}'
    return problem,answer

  elif difficulty == 3:
    case=randint(1,2)
    if case == 1:
      front = randint(2,5)
      numerator1 = randint(1,7)
      denominator1 = randint(2,7)
      numerator2 = randint(1,5)
      denominator2 = randint(2,5)
      start= front*Rational(numerator1,denominator1)
      diff= Rational(numerator2,denominator2)
      arith=[]
      arith.append(sympy.simplify(start))
      for i in range(4):
        arith.append(sympy.simplify(arith[i] + diff))
    if case == 2:
      front = randint(2,5)
      numerator1 = randint(1,7)
      denominator1 = randint(2,7)
      numerator2 = randint(-5,-1)
      denominator2 = randint(2,5)
      start= front * Rational(numerator1,denominator1)
      diff= Rational(numerator2,denominator2)
      arith=[]
      arith.append(sympy.simplify(start))
      for i in range(4):
        arith.append(sympy.simplify(arith[i] + diff))

    if expr == 'latex':
      latexify=[]
      for iv in range(len(arith)):
        num , dem = arith[iv].as_numer_denom()
        front= num//dem
        top = num%dem
        bot = dem
        if top == 0:
          lat= str(front)
        elif front == 0:
          lat= r'\frac{'+str(top)+'}{'+str(bot)+'}'
        else:
          lat= str(front)+r'\frac{'+str(top)+'}{'+str(bot)+'}'
        latexify.append(lat)
      problem = r'$\text{Sequence:} '+fr'{latexify[0]} , {latexify[1]}, {latexify[2]} , {latexify[3]}, {latexify[4]}$'
      if diff > 0:
        answer = '$'+r'f(n)=f(n-1)+'+sympy.latex(sympy.simplify(diff))+'$, $f(1)='+f'{latexify[0]}'+'$'
      else:
        answer = '$'+r'f(n)=f(n-1)'+sympy.latex(sympy.simplify(diff))+'$, $f(1)='+f'{latexify[0]}'+'$'
      return problem,answer
    else:
      problem = r'\text{Sequence:} '+fr'{arith[0]} , {arith[1]}, {arith[2]} , {arith[3]}, {arith[4]}'
      answer = sympy.simplify(diff)
      return problem,answer

for i in range(5):
  a , b = Arithmetic_Sequence_Recursive(1,"latex")
  print(a," common diff: ",b)

for i in range(5):
  a , b = Arithmetic_Sequence_Recursive(2,"latex")
  print(a," common diff: ",b)
for i in range(50):
  a , b = Arithmetic_Sequence_Recursive(3,"latex")
  print(a," common diff: ",b,r"\newline")


$\text{Sequence:} 9 , 17, 25 , 33, 41$  common diff:  $f(n)=f(n-1)+8$, $f(1)=9$
$\text{Sequence:} 6 , 8, 10 , 12, 14$  common diff:  $f(n)=f(n-1)+2$, $f(1)=6$
$\text{Sequence:} 5 , 12, 19 , 26, 33$  common diff:  $f(n)=f(n-1)+7$, $f(1)=5$
$\text{Sequence:} 10 , 13, 16 , 19, 22$  common diff:  $f(n)=f(n-1)+3$, $f(1)=10$
$\text{Sequence:} -3 , -7, -11 , -15, -19$  common diff:  $f(n)=f(n-1)-4$, $f(1)=-3$
$\text{Sequence:} 1 , 0, -1 , -2, -3$  common diff:  $f(n)=f(n-1)-1$, $f(1)=1$
$\text{Sequence:} \frac{3}{2} , 1, \frac{1}{2} , 0, - \frac{1}{2}$  common diff:  $f(n)=f(n-1)- \frac{1}{2}$, $f(1)=3/2$
$\text{Sequence:} \frac{1}{5} , - \frac{22}{15}, - \frac{47}{15} , - \frac{24}{5}, - \frac{97}{15}$  common diff:  $f(n)=f(n-1)- \frac{5}{3}$, $f(1)=1/5$
$\text{Sequence:} \frac{2}{3} , - \frac{7}{12}, - \frac{11}{6} , - \frac{37}{12}, - \frac{13}{3}$  common diff:  $f(n)=f(n-1)- \frac{5}{4}$, $f(1)=2/3$
$\text{Sequence:} \frac{3}{5} , - \frac{3}{20}, - \frac{9}{10} , - \frac{33}{20}, - \fra

### Section 3
#### instruction : Find the next 4 terms in each sequence. Identify the explicit equation. 

- easy
  - case 1
  - problem : If the first two terms of an arithmetic sequence are $3$ and $9$ what would the next four numbers be? and What would be the explicit equation for the sequence?
$$\text{Sequence}: 3, 9 , \text{__} ,\text{__} ,\text{__}  ,\text{__}$$
  - answer : $ \text{Sequence}: 3,9,15,21,27$
  - $ f(n)=3+6(n-1) $ or $ f(n) = 6n-3$
  - case 2
  - problem : If the first two terms of an arithmetic sequence are $3$ and $1$ what would the next four numbers be? and What would be the explicit equation for the sequence?
$$\text{Sequence}: 3, 1 , \text{__} ,\text{__} ,\text{__}  ,\text{__}$$
  - answer : $ \text{Sequence}: 3,1,-1,-3,-5$
  - $ f(n)=3-2(n-1) $ or $ f(n) = -2n+5$
- medium
  - case 1
  - problem : If the first two terms of an arithmetic sequence are $\frac{1}{2}$ and $1$ what would the next four numbers be? and What would be the explicit equation for the sequence?
  $$ \text{Sequence}: \frac{1}{2},1, \text{__} ,\text{__} ,\text{__}  ,\text{__}$$
  - answer : $f(n)=\frac{1}{2}+\frac{1}{2}(n-1)$ , $ f(n) = \frac{1}{2}n$
  - case 2
  - problem : problem : If the first two terms of an arithmetic sequence are $\frac{1}{2}$ and $\frac{5}{6}$ what would the next four numbers be? and What would be the explicit equation for the sequence?
  $$ \text{Sequence}: -\frac{1}{2},-\frac{5}{6}, \text{__} ,\text{__} ,\text{__}  ,\text{__}$$
  $$ \text{Sequence}: -\frac{1}{2},-\frac{5}{6},-\frac{7}{6},-\frac{3}{2},-\frac{11}{6}$$
  - answer : $f(n)=-\frac{1}{2}(n-1) +\frac{1}{3}$ , $-\frac{1}{2}$
- hard
  - case 1
  - problem : What is the the common difference?
  $$ \text{Sequence}: 3\frac{1}{2},4,4\frac{1}{2},5,5\frac{1}{2}$$
  -  answer : $f(n)=f(n-1) -\frac{1}{2}$ , $3\frac{1}{2}$
  - case 2
  - problem : What is the the common difference?
  $$ \text{Sequence}: 3\frac{1}{3},1,-1\frac{1}{3},-3\frac{2}{3},-6$$
  - answer : $ f(n)=f(n-1) -\frac{7}{3}$ , $ 3\frac{1}{3}$


In [ ]:
for i in range(10):
    problem, answer = Arithmetic_Sequence_Problem(3, 'explicit', blanks=True)
    print(problem, r'\\')
    print(answer, r'\\ \\')

$- \frac{2}{5}, - \frac{1}{15}, \underline{\hspace{4mm}}, \underline{\hspace{4mm}}, \underline{\hspace{4mm}}, \underline{\hspace{4mm}}, \ldots$ \\
$f(n) = \frac{1}{3}\left(n-1\right)- \frac{2}{5} \quad\text{or}\quad f(n) = \frac{1}{3}n- \frac{11}{15}$ \newline $- \frac{2}{5}, - \frac{1}{15}, \frac{4}{15}, \frac{3}{5}, \frac{14}{15}, \frac{19}{15}, \ldots$ \\ \\
$\frac{1}{3}, 2, \underline{\hspace{4mm}}, \underline{\hspace{4mm}}, \underline{\hspace{4mm}}, \underline{\hspace{4mm}}, \ldots$ \\
$f(n) = \frac{5}{3}\left(n-1\right) + \frac{1}{3} \quad\text{or}\quad f(n) = \frac{5}{3}n- \frac{4}{3}$ \newline $\frac{1}{3}, 2, \frac{11}{3}, \frac{16}{3}, 7, \frac{26}{3}, \ldots$ \\ \\
$\frac{5}{3}, 2, \underline{\hspace{4mm}}, \underline{\hspace{4mm}}, \underline{\hspace{4mm}}, \underline{\hspace{4mm}}, \ldots$ \\
$f(n) = \frac{1}{3}\left(n-1\right) + \frac{5}{3} \quad\text{or}\quad f(n) = \frac{1}{3}n + \frac{4}{3}$ \newline $\frac{5}{3}, 2, \frac{7}{3}, \frac{8}{3}, 3, \frac{10}{3}, \ldots$ \

## 1.3.3 Geometric Sequence



In [ ]:
def Geometric_Sequence_Problem(difficulty=1, prob='common', blanks=False, expr="latex"):
    if difficulty == 1: # easy
        if randint(0,1):
            geo = GeoSeq(randint(1,10), [1,randint(1,10)])
        else:
            geo = GeoSeq(randint(-10,-1), [1,randint(-10,-1)])
    
    elif difficulty == 2: # medium
        ratio = Rational(randint(1,7),randint(2,7))
        power = randint(2,9 - ratio.q)
        start = ratio.p*ratio.q**power
        if randint(0,1):
            geo = GeoSeq(ratio, [1,start])
        else:
            geo = GeoSeq(-ratio, [1,start])

    elif difficulty == 3: # hard
        if randint(0,1):
            geo = GeoSeq(Rational(randint(1,5),randint(2,5)), [1,Rational(randint(1,7),randint(2,7))])
        else:
            geo = GeoSeq(Rational(randint(-5,-1),randint(2,5)), [1,Rational(randint(1,7),randint(2,7))])
    
    nums = [1,2,'','','',''] if blanks else [1,2,3,4,5]
    problem = signify(geo.getSeqStr(nums))
    answer = getSeqAnswer(geo, prob, blanks, expr)
            
    return problem, answer

###Section 1
#### instruction : Use the given function to determine a pattern and complete the table.
---

- easy


- medium
  - problem :
  - answer : 
- hard
  - problem : If the first two terms of a geometric sequence are $3$ and $9$ what would the next four numbers be? and What would be the explicit equation for the sequence?
$$\text{Sequence}: 3, 9 , \text{__} ,\text{__} ,\text{__}  ,\text{__}   … 
$$
  - answer : $ \text{Sequence}: 3,9,27,81,243,723$
  - $ f(n)=3(3)^{n-1} $ or $ f(n) =3^{n}$

In [ ]:
for i in range(10):
    problem, answer = Geometric_Sequence_Problem(2, 'common')
    print(problem, r'\\')
    print(answer, r'\\ \\')

$448, -784, 1372, -2401, \frac{16807}{4}, \ldots$ \\
$r = - \frac{7}{4}$ \\ \\
$1, -1, 1, -1, 1, \ldots$ \\
$r = -1$ \\ \\
$128, 64, 32, 16, 8, \ldots$ \\
$r = \frac{1}{2}$ \\ \\
$75, 45, 27, \frac{81}{5}, \frac{243}{25}, \ldots$ \\
$r = \frac{3}{5}$ \\ \\
$3750, 4500, 5400, 6480, 7776, \ldots$ \\
$r = \frac{6}{5}$ \\ \\
$1, -1, 1, -1, 1, \ldots$ \\
$r = -1$ \\ \\
$768, -576, 432, -324, 243, \ldots$ \\
$r = - \frac{3}{4}$ \\ \\
$4, 2, 1, \frac{1}{2}, \frac{1}{4}, \ldots$ \\
$r = \frac{1}{2}$ \\ \\
$320, -400, 500, -625, \frac{3125}{4}, \ldots$ \\
$r = - \frac{5}{4}$ \\ \\
$896, 3136, 10976, 38416, 134456, \ldots$ \\
$r = \frac{7}{2}$ \\ \\


### Section 2
#### Find the recursive equation.


In [ ]:
for i in range(10):
    problem, answer = Geometric_Sequence_Problem(2, 'recursive')
    print(problem, r'\\')
    print(answer, r'\\ \\')

$1, 1, 1, 1, 1, \ldots$ \\
$f(n) = 1\cdot f(n-1),\quad f(1) = 1 \quad\text{or}\quad f(n) = 1\cdot f(n-1),\quad f(0) = 1$ \\ \\
$27, -9, 3, -1, \frac{1}{3}, \ldots$ \\
$f(n) = - \frac{1}{3}\cdot f(n-1),\quad f(1) = 27 \quad\text{or}\quad f(n) = - \frac{1}{3}\cdot f(n-1),\quad f(0) = -81$ \\ \\
$1, -1, 1, -1, 1, \ldots$ \\
$f(n) = -1\cdot f(n-1),\quad f(1) = 1 \quad\text{or}\quad f(n) = -1\cdot f(n-1),\quad f(0) = -1$ \\ \\
$2, -4, 8, -16, 32, \ldots$ \\
$f(n) = -2\cdot f(n-1),\quad f(1) = 2 \quad\text{or}\quad f(n) = -2\cdot f(n-1),\quad f(0) = -1$ \\ \\
$7168, -12544, 21952, -38416, 67228, \ldots$ \\
$f(n) = - \frac{7}{4}\cdot f(n-1),\quad f(1) = 7168 \quad\text{or}\quad f(n) = - \frac{7}{4}\cdot f(n-1),\quad f(0) = -4096$ \\ \\
$375, -225, 135, -81, \frac{243}{5}, \ldots$ \\
$f(n) = - \frac{3}{5}\cdot f(n-1),\quad f(1) = 375 \quad\text{or}\quad f(n) = - \frac{3}{5}\cdot f(n-1),\quad f(0) = -625$ \\ \\
$98, 28, 8, \frac{16}{7}, \frac{32}{49}, \ldots$ \\
$f(n) = \frac{2}{7}\cdot f(n-1),

### Section 3
#### instruction : Find the next 4 terms in each sequence. Identify the explicit equation. 

- easy
  - problem : If the first two terms of an arithmetic sequence are $3$ and $9$ what would the next four numbers be? and What would be the explicit equation for the sequence?
$$\text{Sequence}: 3, 9 , \text{__} ,\text{__} ,\text{__}  ,\text{__}$$
  - answer : $ \text{Sequence}: 3,9,15,21,27$
  - $ f(n)=3+6(n-1) $ or $ f(n) = 6n-3$
- medium

- hard



In [ ]:
for i in range(10):
    problem, answer = Geometric_Sequence_Problem(2, 'explicit', blanks=True)
    print(problem, r'\\')
    print(answer, r'\\ \\')

$2, 4, \underline{\hspace{4mm}}, \underline{\hspace{4mm}}, \underline{\hspace{4mm}}, \underline{\hspace{4mm}}, \ldots$ \\
$f(n) = 2 \cdot 2^{n - 1} \quad\text{or}\quad f(n) = 2^{n}$ \newline $2, 4, 8, 16, 32, 64, \ldots$ \\ \\
$216, -36, \underline{\hspace{4mm}}, \underline{\hspace{4mm}}, \underline{\hspace{4mm}}, \underline{\hspace{4mm}}, \ldots$ \\
$f(n) = 216\left(- \frac{1}{6}\right)^{n - 1} \quad\text{or}\quad f(n) = -1296\left(- \frac{1}{6}\right)^{n}$ \newline $216, -36, 6, -1, \frac{1}{6}, - \frac{1}{36}, \ldots$ \\ \\
$1458, 972, \underline{\hspace{4mm}}, \underline{\hspace{4mm}}, \underline{\hspace{4mm}}, \underline{\hspace{4mm}}, \ldots$ \\
$f(n) = 1458\left(\frac{2}{3}\right)^{n - 1} \quad\text{or}\quad f(n) = 2187\left(\frac{2}{3}\right)^{n}$ \newline $1458, 972, 648, 432, 288, 192, \ldots$ \\ \\
$147, 63, \underline{\hspace{4mm}}, \underline{\hspace{4mm}}, \underline{\hspace{4mm}}, \underline{\hspace{4mm}}, \ldots$ \\
$f(n) = 147\left(\frac{3}{7}\right)^{n - 1} \quad\text

## 1.3.4 So, Should We Use Recursive Or Explicit?

### section 1 
#### create the missing equation for the sequence based on the table, the description, or the given equation. Then decide which equation will help to find the missing term of the sequence most efficiently. Find the term and explain your choice.

Case 1:

Recursive : $\text{now} = \text{previous term} + 3$

| term # | 1 | 2 | 3 | 4 |
| :--- | :--- | :--- | :--- | :--- |
| value | 10 | 13 | 16 |  | 

- a) Explicit Equation : 
- b) Find the value of $\text{4th}$ term : 
- c) Explanation : 

Case 2:

Explicit : $f(x) = 4x+6$

| term # | 1 | 2 | ... | 50 |
| :--- | :--- | :--- | :--- | :--- |
| value | 10 | 14 | ... |  | 

- a) Recursive Equation : 
- b) Find the value of $\text{50th}$ term : 
- c) Explanation : 

Case 3:

The value of the $\text{2nd}$ term is $24$. The sequence is being $\text{doubled}$ at each step Explicit equation: $f(n)=6(2)^n$

- a) Recursive Equation : 
- b) Find the value of $\text{2nd}$ term : 
- c) Explanation : 

Case 4:

The value of the $\text{3rd}$ term is $12$. The sequence is being doubled at each step. Recursive: $\text{now} = \text{previous} \times 2$

- a) Explicit Equation : 
- b) Find the value of $\text{10th}$ term : 
- c) Explanation : 





In [ ]:

#1.3.4 So, Should We Use Recursive Or Explicit?

# absolute path of the parent directory to the sys.path

import os, sys, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

from loader import ArithSeq, getInt, GeoSeq, latexify, signify, tableGenerator
from sympy import Rational, latex
from random import randint


def getSequenceDescription(seq, given=4):
    ratioDescrips = {2: 'doubled', 3: 'tripled', 4: 'quadrupled', Rational(1,2): 'halved'}
    
    description = r'The value of the \nth{' + str(given) + '} term is $' + latex(seq.findTerm(given)) + '$.  '
    if type(seq)==ArithSeq:
        changing = 'increasing' if seq.common>=0 else 'decreasing'
        description += fr'The sequence is {changing} by ${latex(abs(seq.common))}$ at each step.'
    elif type(seq)==GeoSeq:
        if seq.common in ratioDescrips.keys():
            change = ratioDescrips[seq.common]
        else:
            change = fr'multiplied by ${latex(seq.common)}$'
        description += f'The sequence is being {change} at each step.'

    return description

def So_Should_We_Use_Recursive_Or_Explicit(difficulty=1, expr='latex'):

    if difficulty == 1: # easy
        common = randint(2, 5)
        start = randint(1, 10)
    elif difficulty == 2: # medium
        common = getInt(-5, 5, exclude=[-1,0,1])
        start = getInt(-5, 5)
    elif difficulty == 3: # hard
        common = Rational(getInt(-5,5), randint(2,5))
        start = Rational(getInt(-5,5), randint(2,5))

    if randint(0,1):
        seq = ArithSeq(common, [1,start])
    else:
        seq = GeoSeq(common, [1,start])
    
    useExplicit = randint(0,1)

    if randint(0,1):
        if useExplicit:
            termNum = randint(30,80) if type(seq)==ArithSeq else randint(5,9)
            header = [r'term \#', 1, 2, r'$\ldots$', termNum]
            row = ['value', signify(latexify(seq.terms[1])), signify(latexify(seq.terms[2])), r'$\ldots$', '']
        else:
            termNum = 4
            header = [r'term \#', 1, 2, 3, 4]
            row = ['value', signify(latexify(seq.terms[1])), signify(latexify(seq.terms[2])), signify(latexify(seq.terms[3])), '']
        problem = tableGenerator(header, [row])
    else:
        givenNum = randint(2,4)
        if useExplicit:
            termNum = randint(30,80) if type(seq)==ArithSeq else randint(6,10)
        else:
            termNum = givenNum+randint(1,2)
        problem = getSequenceDescription(seq, given=givenNum) + r' \newline ' 
    
    problem += r'Explicit equation: $' + seq.getExplicit(0) + r'$ \newline '
    problem += r'Recursive equation: $\text{now} = \text{previous}'
    change = latexify(seq.common, seq.precision)
    if type(seq) == ArithSeq:
        problem += r' + ' if seq.common>0 else ''
    else:
        problem += r' \cdot '
        change = '(' + change +')'if seq.common<0 else change
    problem += change + r'$ \newline '

    problem += r'Find the value of the \nth{' + str(termNum) + '} term.'

    answer = seq.findTerm(termNum)
    answer = signify(latexify(answer, seq.precision)) if expr=='latex' else answer

    return problem, answer

for jj in range(10):
    problem, answer = So_Should_We_Use_Recursive_Or_Explicit(2)
    print(problem, answer)

ModuleNotFoundError: ignored

# Archive

In [ ]:
def getGeo(start, ratio, num):
  compiledlist=[]
  current_term=start
  for i in range(0, num):
    compiledlist.append(current_term)
    current_term *= ratio
  return compiledlist

  |#----- generator
# Formula: 
# Arithmetic Sequence Formula: end = start + (n - 1)d
# end - start = (n - 1)d
# (end - start) / d = n - 1
# (end - start) / d + 1 = n 

def find_term_n(start, n, diff):
    if start % 1 != 0:
        print("Please choose an integer.")
    elif n % 1 != 0 and n < 1:
        print("Please choose a positive whole number.")
    else:
        term_n = start + (n - 1)*diff
        return term_n
def find_numTerms_arithSeq(start, end, diff):
    if start % 1 != 0:
        print("Please choose an integer.")
    numTerms = 1 + (end - start) / diff
    if numTerms > 0:
        return int(numTerms)
    else:
        print("The number of terms cannot be negative. Try again.")
def is_arithmetic(arithlist):
    delta = arithlist[1] - arithlist[0]
    for index in range(len(l) - 1):
        if not (arithlist[index + 1] - arithlist[index] == delta):
             return False
    return True

def find_term_n_geo(start, n, ratio):
    if start % 1 != 0:
        print("Please choose an integer.")
    elif n % 1 != 0 and n < 1:
        print("Please choose a positive whole number.")
    else:
        return start*pow(ratio,n-1)

def find_numTerms_geoSeq(start, end, ratio):
    if start % 1 != 0:
        print("Please choose an integer.")
    numTerms = 1 + log(end/start,ratio)
    if numTerms > 0:
        return int(numTerms)
    else:
        print("The number of terms cannot be negative. Try again.")

def is_geometric(li):
    if len(li) <= 1:
        return True
    # Calculate ratio
    ratio = li[1]/float(li[0])
    # Check the ratio of the remaining
    for i in range(1, len(li)):
        if li[i]/float(li[i-1]) != ratio: 
            return False
    return True 

def printGP(start, ratio, num):
    for i in range(0, num):
        curr_term = start * pow(ratio, i)
        print(curr_term, end =" ")
    return geoList

In [ ]:
"""Section 2
instruction : Determine the y-value of each ordered pair based on the given x- value.
"""
import random
import sympy

def floatA_f():
    floatA = random.choice([0.3 , 0.5 , 0.9]) #[ 0.3 , 0.5 , 0.9]

def floatB_f():
    floatB = random.choice(["-", ""]) + str(random.randint(1,9)) + "." + str(random.randint(1,99)) #[ -9.9,9.9]

def intA_f():
    intA = random.choice([-4,-3,-2,-1,1,2,3,4])
    return intA

def intB_f():
    intB = random.choice([-4,-3,-2,-1,1,2,3,4])
    return intB

def intC_f():
    intC = random.choice([-4,-3,-2,-1,1,2,3,4])
    return intC

def intD_f():
    intD = random.choice([-4,-3,-2,-1,1,2,3,4])
    return intD

def intE_f():
    intE = random.choice([-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,1,2,3,4,5,6,7,8,9,10])
    return intE

def intF_f():
    intF = random.choice([-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,1,2,3,4,5,6,7,8,9,10])
    return intF

def Evaluate_Equations_Section_2(option_difficulty="easy", expr="latex"):
    problem = ""
    answer = ""
    variables = random.choice(['a', 'b', 'z', 'p', 't', 'q', 'k', 'u', 'r', 'd', 'w', 's', 'h', 'v'])
    
    front = ""
    other = ""
    if option_difficulty == "easy":
        """easy
        case 1
        problem :  f(x)=3x+7
        answer :  (−3,),(2,),(5,)
        case 2
        problem :  f(x)=13x+7
        answer :  (−3,),(2,),(5,)"""
        intA = intA_f()
        intB = intB_f() 
        intC = intC_f() 
        intD = intD_f()
        intE = intE_f()
        intF = intF_f()
        if random.randint(1,2) == 1:
            front = fr'{variables}(x)'
            other = fr'{intA} * x + {intB}' #intA_f
            problem = fr'({intD}), ({intE}), ({intF})'
        else:
            front = fr'{variables}(x)'
            other = fr'{intA} / {intE} * x + {intF}' #intE_f
            problem = fr'({intD}), ({intC}), ({intF})'

        if random.randint(1,2) == 1:
            front = fr'y'
        

        """medium
        case 1
        problem :  f(x)=3(6m−17)
        answer :  (−3,),(2,),(5,)
        case 2
        problem :  f(x)=2(x+10)3
        answer :  (−3,),(2,),(5,)
        """

        """        hard
        case 1
        problem :  f(x)=2x2+3x+5
        answer :  (−3,),(2,),(5,)
        case 2
        problem :  f(x)=2x2+3x+5
        answer :  (−3,),(2,),(5,)"""
    return()
"""easy
case 1
problem : inA*x + intB = intC (x = intD; x = intE ; x = intF)
answer :
problem :  3x+2=−4  ( x=3;x=2;x=−2 )
answer : x = -2
case 2
problem : inAx + intBx = intC (x = intD; x = intE ; x = intF)
answer :
problem :  2x+x=−3  ( x=3;x=2;x=−2 )
answer : x = -1
medium
case 1
problem : inAx + intBx + intC = intD (x = ; x = ; x = )
answer : x =
problem : 8x + 7x + 3 = 13 (x = -3; x = 2 ; x = 5)
answer x = , ,
case 2
problem : inA(x + intB) + intC = intD (x = ; x = ; x = )
answer : x =
problem : 2(x + 3) = 5 (x = -3; x = 2 ; x = 5)
answer x = , ,
hard
case 1
problem : intA(intBx + intCx) + intD = intE (x = ; x = ; x = )
answer : x =
problem : 2(x + 7) + 3 = 5 (x = -3; x = 2 ; x = 5)
answer x = , ,
case 2
problem : inAx + intBx + intC = intD (x = ; x = ; x = )
answer : x =
problem :  3(6m−17)=13(x=−3;x=2;x=5) 
answer x = , ,"""

print("easy")
for i in range(3):
    problem, answer = Evaluate_Equations_Section_2("easy", "latex")
    print(problem, answer)

In [ ]:
"""Section 3
instruction : Given that  f(x)=8x−3  and  g(x)=3x−10 , evaluate the following functions with the indicated values.
medium
case 1
problem : linear equation
f(x) = intA * x + intB
g(x) = intA * x + intB
f(5)  =
g(5)  =
case 2
f(x) = intA * x + intB
g(x) = intA * x + intB
f(5)=
g(5)=
hard
case 1
problem : linear equation
f(x) = intA * x + intB
g(x) = intA * x + intB
f(5)  =
g(5)  =
case 2
f(x) = intA * x + intB
g(x) = intA * x + intB
f(5)=
g(5)="""
import random
import sympy

def floatA_f():
    floatA = random.choice([0.3 , 0.5 , 0.9]) #[ 0.3 , 0.5 , 0.9]

def floatB_f():
    floatB = random.choice(["-", ""]) + str(random.randint(1,9)) + "." + str(random.randint(1,99)) #[ -9.9,9.9]

def intA_f(var = False):
    if var == True:
        return str(random.choice(["", "-"])) + str(random.randint(1, 4))
    intA = random.randint(-4, 4) #[-4,-3,-2,-1,1,2,3,4]
    return intA

def intB_f(var = False):
    if var == True:
        return str(random.choice(["", "-"])) + str(random.randint(1, 4))
    intB = random.randint(-4,4) #[-4,-3,-2,-1,1,2,3,4]
    return intB

def intC_f(var = False):
    if var == True:
        return str(random.choice(["", "-"])) + str(random.randint(1, 4))

    intC = random.randint(-4,4) #[-4,-3,-2,-1,1,2,3,4]
    return intC

def intD_f(var = False):
    if var == True:
        return str(random.choice(["", "-"])) + str(random.randint(1, 4))

    intD = random.randint(-4,4) #[-4,-3,-2,-1,1,2,3,4]
    return intD

def intE_f(var = False):
    if var == True:
        return str(random.choice(["", "-"])) + str(random.randint(1, 10))

    intE = random.randint(-10,10) #[-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,1,2,3,4,5,6,7,8,9,10]
    return intE

def intF_f(var = False):
    if var == True:
        return str(random.choice(["", "-"])) + str(random.randint(1, 10))

    intF = random.randint(-10,10) #[-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,1,2,3,4,5,6,7,8,9,10]
    return intF

def eq_header_gen():
    if random.randint(1,2):
        return "f(x)"

    else:
        return "g(x)"

def function(option_difficulty = "easy", expr = "latex"):
    """easy
    case 1
    problem : linear equation
    f(x) = intA * x + intB
    g(x) = intA * x + intB
    f(5)  =
    g(5)  =
    case 2
    f(x) = intA * x + intB
    g(x) = intA * x + intB
    f(5)=
    g(5)="""
    sym_1 = random.choice(["-", "+"])

    problem = ""
    answer = ""
    front = ""
    other = ""

    intA = intA_f()
    intB = intB_f()
    intC = intC_f()

    intD = intD_f()

    if option_difficulty == "easy":
        x = sympy.Symbol("x")
        front = eq_header_gen()
        other = fr'{intA} * x {sym_1} {intB}'

    problem = front + " = " + other
    answer = sympy.solve(sympy.sympify(other.replace("x", str(intC))))
    print(problem)
    print(sympy.sympify(other.replace("x", str(intC))))
    print(problem)
    answer = front.replace(")", str(intC) + ")") + " = " + answer.pop(0)
    return(problem, answer)

print(function("easy", "latex"))